In [1]:
# connect collab to google drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
# print where you base repertory is
import os
print (os.getcwd())

/content


In [3]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

# 0 - Verification (run the A section every session)


## A - needed to run

In [4]:
# see what's file in the projetc
!ls /content/gdrive/MyDrive/wound_management

basic_wound_management_yolov11n-seg.pt	data_with_ref
basic_wound_management_yolov11x-seg.pt	data.yaml
basic_wound_model_yolov11n-seg.pt	healing
basic_wound_model_yolov11x-seg.pt	output
data					wound_management_deeplabv3.pth
data1.yaml				wound_management.ipynb
data2.yaml				wound_management_yolov11n-seg.pt
data_buff				wound_model_deeplabv3.pth
data_pytorch_1.yaml			wound_model_yolov11n-seg.pt
data_pytorch.yaml


In [5]:
!rm /content/gdrive/MyDrive/wound_management/data/train/labels.cache
!rm /content/gdrive/MyDrive/wound_management/data/val/labels.cache

# Verify the cache is deleted
!ls /content/gdrive/MyDrive/wound_management/data/train

rm: cannot remove '/content/gdrive/MyDrive/wound_management/data/train/labels.cache': No such file or directory
rm: cannot remove '/content/gdrive/MyDrive/wound_management/data/val/labels.cache': No such file or directory
images	labels


In [6]:
!rm /content/gdrive/MyDrive/wound_management/data_with_ref/train/labels.cache
!rm /content/gdrive/MyDrive/wound_management/data_with_ref/val/labels.cache

# Verify the cache is deleted
!ls /content/gdrive/MyDrive/wound_management/data_with_ref/train

rm: cannot remove '/content/gdrive/MyDrive/wound_management/data_with_ref/train/labels.cache': No such file or directory
rm: cannot remove '/content/gdrive/MyDrive/wound_management/data_with_ref/val/labels.cache': No such file or directory
images	labels	masks


In [7]:
!rm /content/gdrive/MyDrive/wound_management/data_buff/train/labels.cache
!rm /content/gdrive/MyDrive/wound_management/data_buff/val/labels.cache

# Verify the cache is deleted
!ls /content/gdrive/MyDrive/wound_management/data_buff/train

rm: cannot remove '/content/gdrive/MyDrive/wound_management/data_buff/train/labels.cache': No such file or directory
rm: cannot remove '/content/gdrive/MyDrive/wound_management/data_buff/val/labels.cache': No such file or directory
images	labels	masks


In [8]:
import matplotlib.pyplot as plt
from ultralytics import YOLO
from pathlib import Path
import shutil
import ultralytics
import numpy as np
from IPython.display import clear_output
import torch
import cv2
import math
from google.colab.patches import cv2_imshow
from PIL import Image, ImageDraw
from pathlib import Path
import torchvision.transforms as T
from torchvision import transforms
import yaml
import torch.nn as nn
import torch.nn.functional as F
import timm
import time
import torch.optim as optim
from scipy.ndimage import binary_dilation
from torch.utils.data import Dataset, DataLoader
import torchvision.models.segmentation as seg_models

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## B - mask to label

In [ ]:
"""Convert training's mask to training's label"""

input_dir = "/content/gdrive/MyDrive/wound_management/data_buff/train/masks"
output_dir = "/content/gdrive/MyDrive/wound_management/data_buff/train/labels"

for j in os.listdir(input_dir):
    image_path = os.path.join(input_dir, j)

    mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    H, W = mask.shape

    # get contours
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # convert the contours to polygons
    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:
            polygon = []
            for point in cnt:
                x, y = point[0]
                polygon.append(x/W)
                polygon.append(y/H)
            polygons.append(polygon)

    # print the polygons
    with open('{}.txt'.format(os.path.join(output_dir, j)[:-4]), 'w') as f:
        for polygon in polygons:
            for p_, p in enumerate(polygon):
                if p_ == len(polygon) - 1:
                    f.write('{}\n'.format(p))
                elif p_ == 0:
                    f.write('0 {} '.format(p))
                else:
                    f.write('{} '.format(p))

        f.close()

In [ ]:
""" Convert val's mask to val's label"""

input_dir = "/content/gdrive/MyDrive/wound_management/data_buff/val/masks"
output_dir = "/content/gdrive/MyDrive/wound_management/data_buff/val/labels"

for j in os.listdir(input_dir):
    image_path = os.path.join(input_dir, j)

    mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    H, W = mask.shape

    # get contours
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # convert the contours to polygons
    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:
            polygon = []
            for point in cnt:
                x, y = point[0]
                polygon.append(x/W)
                polygon.append(y/H)
            polygons.append(polygon)

    # print the polygons
    with open('{}.txt'.format(os.path.join(output_dir, j)[:-4]), 'w') as f:
        for polygon in polygons:
            for p_, p in enumerate(polygon):
                if p_ == len(polygon) - 1:
                    f.write('{}\n'.format(p))
                elif p_ == 0:
                    f.write('0 {} '.format(p))
                else:
                    f.write('{} '.format(p))

        f.close()

## C - Label to mask

In [ ]:
""" Convert label to mask"""

def yolo_to_mask_ref(txt_path, image_path, output_mask_path):
    # Load the image and know her size
    with Image.open(image_path) as img:
        w, h = img.size

    # Create a black image
    mask = Image.new('RGB', (w, h), (0, 0, 0))
    draw = ImageDraw.Draw(mask)

    colors = {
        0: (0, 0, 255),   # ref: Blue
        1: (0, 255, 0),   # wound: Green
    }

    # read the label
    with open(txt_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) < 7 or (len(parts) - 1) % 2 != 0:
            continue

        cls = int(parts[0])
        coords = parts[1:]

        # Polygon usage because we want precise masks
        polygon = []
        for i in range(0, len(coords), 2):
            x_norm = float(coords[i])
            y_norm = float(coords[i+1])
            x_abs = x_norm * w
            y_abs = y_norm * h
            polygon.append((x_abs, y_abs))

        # Get the RGB color for the class, default to white if undefined
        fill_color = colors.get(cls, (255, 255, 255))
        draw.polygon(polygon, fill=fill_color)

    mask.save(output_mask_path)
    print(f"Masque saved : {output_mask_path}")



def yolo_to_mask(txt_path, image_path, output_mask_path):
    # Load the image and know her size
    with Image.open(image_path) as img:
        w, h = img.size

    # Create a black image
    mask = Image.new('RGB', (w, h), (0, 0, 0))
    draw = ImageDraw.Draw(mask)

    colors = {
        0: (0, 255, 0),   # wound: Green
    }

    # read the label
    with open(txt_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) < 7 or (len(parts) - 1) % 2 != 0:
            continue

        cls = int(parts[0])
        coords = parts[1:]

        # Polygon usage because we want precise masks
        polygon = []
        for i in range(0, len(coords), 2):
            x_norm = float(coords[i])
            y_norm = float(coords[i+1])
            x_abs = x_norm * w
            y_abs = y_norm * h
            polygon.append((x_abs, y_abs))

        # Get the RGB color for the class, default to white if undefined
        fill_color = colors.get(cls, (255, 255, 255))
        draw.polygon(polygon, fill=fill_color)

    mask.save(output_mask_path)
    print(f"Masque saved : {output_mask_path}")

In [ ]:
""" Conversion of training's labels in training's masks for the wound"""

input_images_dir = "/content/gdrive/MyDrive/wound_management/data_buff/train/images"
input_labels_dir = "/content/gdrive/MyDrive/wound_management/data_buff/train/labels"
output_masks_dir = "/content/gdrive/MyDrive/wound_management/data_buff/train/masks"

os.makedirs(output_masks_dir, exist_ok=True)

for filename in os.listdir(input_images_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(input_images_dir, filename)
        txt_path = os.path.join(input_labels_dir, os.path.splitext(filename)[0] + ".txt")
        output_mask_path = os.path.join(output_masks_dir, os.path.splitext(filename)[0] + ".png")

        if os.path.exists(txt_path):
            yolo_to_mask(txt_path, image_path, output_mask_path)

Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/train/masks/wsnet_0122.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/train/masks/wsnet_0183.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/train/masks/wsnet_0162.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/train/masks/wsnet_0215.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/train/masks/wsnet_0156.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/train/masks/wsnet_0124.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/train/masks/wsnet_0229.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/train/masks/wsnet_0166.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/train/masks/wsnet_0146.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/train/masks/wsnet_0171.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/trai

In [ ]:
""" Conversion of val's labels in val's masks for the wound"""

input_images_dir = "/content/gdrive/MyDrive/wound_management/data_buff/val/images"
input_labels_dir = "/content/gdrive/MyDrive/wound_management/data_buff/val/labels"
output_masks_dir = "/content/gdrive/MyDrive/wound_management/data_buff/val/masks"

os.makedirs(output_masks_dir, exist_ok=True)

for filename in os.listdir(input_images_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(input_images_dir, filename)
        txt_path = os.path.join(input_labels_dir, os.path.splitext(filename)[0] + ".txt")
        output_mask_path = os.path.join(output_masks_dir, os.path.splitext(filename)[0] + ".png")

        if os.path.exists(txt_path):
            yolo_to_mask(txt_path, image_path, output_mask_path)

Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/val/masks/0577.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/val/masks/0900.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/val/masks/0880.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/val/masks/0948.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/val/masks/0557.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/val/masks/0549.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/val/masks/0940.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/val/masks/0954.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/val/masks/0375.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/val/masks/0644.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff/val/masks/0447.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_buff

In [ ]:
""" Conversion of training's labels in training's masks for the wound and the reference"""

input_images_dir = "/content/gdrive/MyDrive/wound_management/data_with_ref/train/images"
input_labels_dir = "/content/gdrive/MyDrive/wound_management/data_with_ref/train/labels"
output_masks_dir = "/content/gdrive/MyDrive/wound_management/data_with_ref/train/masks"

os.makedirs(output_masks_dir, exist_ok=True)

for filename in os.listdir(input_images_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(input_images_dir, filename)
        txt_path = os.path.join(input_labels_dir, os.path.splitext(filename)[0] + ".txt")
        output_mask_path = os.path.join(output_masks_dir, os.path.splitext(filename)[0] + ".png")

        if os.path.exists(txt_path):
            yolo_to_mask_ref(txt_path, image_path, output_mask_path)

Masque saved : /content/gdrive/MyDrive/wound_management/data_with_ref/train/masks/photo75_jpg.rf.7f9298b3a0556936c6727d870d769530.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_with_ref/train/masks/photo55_jpg.rf.d2a91c98dae41d470bcb815f0ac382b8.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_with_ref/train/masks/photo72_jpg.rf.bd1281141e3e9575ae3fe748273253ac.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_with_ref/train/masks/photo63_jpg.rf.15fcd8b9b2f1cc35d4ce376790358d6b.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_with_ref/train/masks/photo69_jpg.rf.bdef3ce0c1f0340a230aa3a2b2bd3d9a.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_with_ref/train/masks/photo94_jpg.rf.dbf99c4879b1fc5445f05b3b5c01e847.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_with_ref/train/masks/photo87_jpg.rf.6177a7e13ba5cd9a845e6bc34dd11ca2.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_w

In [ ]:
""" Conversion of training's labels in training's masks for the wound and the reference"""

input_images_dir = "/content/gdrive/MyDrive/wound_management/data_with_ref/val/images"
input_labels_dir = "/content/gdrive/MyDrive/wound_management/data_with_ref/val/labels"
output_masks_dir = "/content/gdrive/MyDrive/wound_management/data_with_ref/val/masks"

os.makedirs(output_masks_dir, exist_ok=True)

for filename in os.listdir(input_images_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(input_images_dir, filename)
        txt_path = os.path.join(input_labels_dir, os.path.splitext(filename)[0] + ".txt")
        output_mask_path = os.path.join(output_masks_dir, os.path.splitext(filename)[0] + ".png")

        if os.path.exists(txt_path):
            yolo_to_mask_ref(txt_path, image_path, output_mask_path)

Masque saved : /content/gdrive/MyDrive/wound_management/data_with_ref/val/masks/LUCILAIMG1443_jpg.rf.0acbb4dd9f2f505d8a4bbe6e5c762282.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_with_ref/val/masks/SIMATECIMAG44_jpg.rf.19430d97510c8ca0dd24b033c2dc90d9.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_with_ref/val/masks/SIMATECIMAG28_jpg.rf.1963ca322590a396ade53c68d6bee589.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_with_ref/val/masks/photo31_jpg.rf.c777890445968f854327b0cafa00419d.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_with_ref/val/masks/SIMATECIMAG13_jpg.rf.d03d97a5b97e4ac46fd1bcb4fccd58e0.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_with_ref/val/masks/photo73_jpg.rf.71220ab9536ba9073551c46eff3b7b63.png
Masque saved : /content/gdrive/MyDrive/wound_management/data_with_ref/val/masks/SIMATECIMAG07_jpg.rf.4cfecd8f6428ae045690aadeff836378.png
Masque saved : /content/gdrive/MyDrive/wound_m

## D - Verify the presence of labels and masks

In [ ]:
"""Check the presence of labels and masks"""

def verify_labels_and_masks(images_dir, labels_dir, masks_dir):
    missing_labels = []
    missing_masks = []

    for img in os.listdir(images_dir):
        if img.lower().endswith(('.jpg', '.png')):
            base_name = os.path.splitext(img)[0]
            label_file = base_name + '.txt'

            # check the labels
            if not os.path.exists(os.path.join(labels_dir, label_file)):
                missing_labels.append(label_file)

            # checks the masks
            mask_exists = False
            for ext in ['.png', '.jpg']:
                if os.path.exists(os.path.join(masks_dir, base_name + ext)):
                    mask_exists = True
                    break
            if not mask_exists:
                missing_masks.append(base_name)

    print(f"Labels lost : {len(missing_labels)}")
    print(f"Masks lost : {len(missing_masks)}")

    # if the labels are not here say it
    if missing_labels:
        print(missing_labels)
    else:
        print("All labels are here")

    # if the masks are not here say it
    if missing_masks:
        print(missing_masks)
    else:
        print("All masks are here")

In [ ]:
images_dir = '/content/gdrive/MyDrive/wound_management/data/val/images'
labels_dir = '/content/gdrive/MyDrive/wound_management/data/val/labels'
masks_dir = '/content/gdrive/MyDrive/wound_management/data/val/masks'

print("data/val")
verify_labels_and_masks(images_dir, labels_dir, masks_dir)
# data doesn't get masks and it's fine

data/val
Labels lost : 0
Masks lost : 200
All labels are here
['0006', '0190', '0235', '0089', '0107', '0289', '0030', '0213', '0256', '0301', '0208', '0057', '0046', '0078', '0128', '0203', '0082', '0136', '0007', '0133', '0058', '0146', '0284', '0126', '0122', '0278', '0164', '0233', '0005', '0002', '0025', '0004', '0245', '0085', '0108', '0096', '0177', '0079', '0192', '0124', '0062', '0143', '0033', '0010', '0276', '0303', '0009', '0180', '0134', '0047', '0068', '0291', '0211', '0259', '0201', '0300', '0008', '0262', '0001', '0217', '0251', '0003', '0369', '0607', '0511', '0738', '0365', '0842', '0904', '0437', '0334', '0820', '0470', '0679', '0849', '0533', '0543', '0682', '0728', '0955', '0355', '0363', '0721', '0763', '0652', '0429', '0528', '0503', '0947', '0417', '0946', '0615', '0712', '0745', '0901', '0456', '0835', '0717', '0517', '0781', '0958', '0824', '0971', '0407', '0834', '0867', '0881', '0588', '0541', '0973', '0496', '0483', '0582', '0802', '0985', '0833', '0657', '

In [ ]:
images_dir = '/content/gdrive/MyDrive/wound_management/data/train/images'
labels_dir = '/content/gdrive/MyDrive/wound_management/data/train/labels'
masks_dir = '/content/gdrive/MyDrive/wound_management/data/train/masks'

print("data/train")
verify_labels_and_masks(images_dir, labels_dir, masks_dir)
# data doesn't get masks and it's fine

data/train
Labels lost : 0
Masks lost : 809
All labels are here
['0064', '0017', '0074', '0072', '0065', '0070', '0056', '0027', '0049', '0051', '0022', '0024', '0019', '0045', '0090', '0093', '0040', '0048', '0097', '0035', '0067', '0028', '0011', '0031', '0075', '0039', '0023', '0054', '0012', '0060', '0015', '0080', '0099', '0073', '0020', '0098', '0042', '0016', '0021', '0095', '0086', '0071', '0052', '0013', '0050', '0055', '0083', '0032', '0100', '0018', '0081', '0084', '0059', '0092', '0061', '0014', '0038', '0041', '0088', '0029', '0066', '0026', '0094', '0101', '0077', '0091', '0063', '0036', '0034', '0044', '0087', '0043', '0053', '0037', '0069', '0194', '0421', '0343', '0166', '0167', '0246', '0261', '0275', '0396', '0149', '0312', '0237', '0264', '0151', '0344', '0425', '0197', '0359', '0302', '0216', '0141', '0420', '0138', '0232', '0242', '0335', '0150', '0205', '0272', '0130', '0426', '0230', '0422', '0288', '0371', '0195', '0106', '0340', '0290', '0393', '0110', '0318',

In [ ]:
images_dir = '/content/gdrive/MyDrive/wound_management/data_with_ref/val/images'
labels_dir = '/content/gdrive/MyDrive/wound_management/data_with_ref/val/labels'
masks_dir = '/content/gdrive/MyDrive/wound_management/data_with_ref/val/masks'

print("data_with_ref/val")
verify_labels_and_masks(images_dir, labels_dir, masks_dir)

data_with_ref/val
Labels lost : 0
Masks lost : 0
All labels are here
All masks are here


In [ ]:
images_dir = '/content/gdrive/MyDrive/wound_management/data_with_ref/train/images'
labels_dir = '/content/gdrive/MyDrive/wound_management/data_with_ref/train/labels'
masks_dir = '/content/gdrive/MyDrive/wound_management/data_with_ref/train/masks'

print("data_with_ref/train")
verify_labels_and_masks(images_dir, labels_dir, masks_dir)

data_with_ref/train
Labels lost : 0
Masks lost : 0
All labels are here
All masks are here


In [ ]:
images_dir = '/content/gdrive/MyDrive/wound_management/data_buff/val/images'
labels_dir = '/content/gdrive/MyDrive/wound_management/data_buff/val/labels'
masks_dir = '/content/gdrive/MyDrive/wound_management/data_buff/val/masks'

print("data_buff/val")
verify_labels_and_masks(images_dir, labels_dir, masks_dir)

data_buff/val
Labels lost : 0
Masks lost : 0
All labels are here
All masks are here


In [ ]:
images_dir = '/content/gdrive/MyDrive/wound_management/data_buff/train/images'
labels_dir = '/content/gdrive/MyDrive/wound_management/data_buff/train/labels'
masks_dir = '/content/gdrive/MyDrive/wound_management/data_buff/train/masks'

print("data_buff/train")
verify_labels_and_masks(images_dir, labels_dir, masks_dir)

data_buff/train
Labels lost : 0
Masks lost : 0
All labels are here
All masks are here


# I - yolov11n-seg


## A - yolov11n-seg for detecting wound (basic)

In [ ]:
"""Load a pretrained YOLO11n model"""

model = YOLO("yolo11n-seg.pt")

100%|██████████| 5.90M/5.90M [00:00<00:00, 156MB/s]


In [ ]:
"""Train the model"""

model.train( data="/content/gdrive/MyDrive/wound_management/data.yaml", epochs=250, batch=-1, patience=10)

Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/gdrive/MyDrive/wound_management/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=T

100%|██████████| 755k/755k [00:00<00:00, 25.8MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

 23        [16, 19, 22]  1    683635  ultralytics.nn.modules.head.Segment          [1, 32, 64, [64, 128, 256]]   
YOLO11n-seg summary: 203 layers, 2,842,803 parameters, 2,842,787 gradients, 10.4 GFLOPs

Transferred 510/561 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 115MB/s]


...

                   all        200        221      0.897      0.873      0.923      0.648      0.891      0.869      0.921      0.641
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 47, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



57 epochs completed in 0.405 hours.
Optimizer stripped from runs/segment/train/weights/last.pt, 6.0MB
Optimizer stripped from runs/segment/train/weights/best.pt, 6.0MB

Validating runs/segment/train/weights/best.pt...
Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-seg summary (fused): 113 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:07<00:00,  1.46s/it]


                   all        200        221      0.918      0.864      0.937      0.668      0.909       0.86      0.932      0.659
Speed: 0.4ms preprocess, 4.9ms inference, 0.0ms loss, 4.7ms postprocess per image
Results saved to runs/segment/train


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7bc2b034ed50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 

In [ ]:
"""Record the model"""

detect_path = Path("runs/segment")

# Find the actual train version
train_dirs = sorted(detect_path.glob("train*"), key=os.path.getmtime)

if not train_dirs:
    print(" Aucun dossier 'train*' trouvé dans runs/segment.")
else:
    latest_train_dir = train_dirs[-1]
    best_pt_path = latest_train_dir / "weights" / "best.pt"

    if best_pt_path.exists():
        # Copy it in google drive
        dest_path = "/content/gdrive/MyDrive/wound_management/basic_wound_model_yolov11n-seg.pt"
        shutil.copy(best_pt_path, dest_path)
        print(f"Model copy from : {best_pt_path} in : {dest_path}")
    else:
        print(f"file not found {latest_train_dir}")

Model copy from : runs/segment/train/weights/best.pt in : /content/gdrive/MyDrive/wound_management/basic_wound_model_yolov11n-seg.pt


In [ ]:
"""Validate the model"""

wound_model = YOLO("/content/gdrive/MyDrive/wound_management/basic_wound_model_yolov11n-seg.pt")
wound_model = wound_model.val(data="/content/gdrive/MyDrive/wound_management/data.yaml")

print("mAP50:", wound_model.seg.map50)
print("mAP50-95:", wound_model.seg.map)
print("Precision:", wound_model.seg.p)
print("Recall:", wound_model.seg.r)

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLO11n-seg summary (fused): 113 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 13.9MB/s]


val: Fast image access ✅ (ping: 0.8±0.3 ms, read: 0.6±0.3 MB/s, size: 242.2 KB)


val: Scanning /content/gdrive/MyDrive/wound_management/data/val/labels... 200 images, 22 backgrounds, 0 corrupt: 100%|██████████| 200/200 [01:00<00:00,  3.32it/s]

val: New cache created: /content/gdrive/MyDrive/wound_management/data/val/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [01:36<00:00,  7.41s/it]


                   all        200        221      0.918      0.864      0.937      0.669       0.91       0.86      0.932      0.657
Speed: 12.4ms preprocess, 435.1ms inference, 0.0ms loss, 7.4ms postprocess per image
Results saved to runs/segment/val
mAP50: 0.9318744731390473
mAP50-95: 0.657361455148271
Precision: [    0.90964]
Recall: [    0.85973]


In [ ]:
"""Made a prediction with the model"""

wound_model = YOLO("/content/gdrive/MyDrive/wound_management/basic_wound_model_yolov11n-seg.pt")
# Verify the model is usable
results = wound_model.predict(source="/content/gdrive/MyDrive/wound_management/data/test", conf=0.75)

total_images = len(results)
no_detection_count = 0

for r in results:
    if r.boxes is None or len(r.boxes) == 0:
        no_detection_count += 1
        continue

    for box in r.boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        class_name = wound_model.names[cls_id]

print(f"pourcentage of no detection at 75% : {no_detection_count / total_images:.2%}")


image 1/200 /content/gdrive/MyDrive/wound_management/data/test/1011.png: 640x640 1 wound, 276.5ms
image 2/200 /content/gdrive/MyDrive/wound_management/data/test/1014.png: 640x640 1 wound, 277.6ms
image 3/200 /content/gdrive/MyDrive/wound_management/data/test/1016.png: 640x640 (no detections), 460.2ms
image 4/200 /content/gdrive/MyDrive/wound_management/data/test/1018.png: 640x640 1 wound, 284.3ms
image 5/200 /content/gdrive/MyDrive/wound_management/data/test/1020.png: 640x640 (no detections), 274.7ms
image 6/200 /content/gdrive/MyDrive/wound_management/data/test/1021.png: 640x640 (no detections), 280.7ms
image 7/200 /content/gdrive/MyDrive/wound_management/data/test/1022.png: 640x640 (no detections), 268.7ms
image 8/200 /content/gdrive/MyDrive/wound_management/data/test/1024.png: 640x640 (no detections), 279.6ms
image 9/200 /content/gdrive/MyDrive/wound_management/data/test/1028.png: 640x640 1 wound, 265.9ms
image 10/200 /content/gdrive/MyDrive/wound_management/data/test/1029.png: 640

In [ ]:
"""Make a prediction and save the results"""

results = wound_model.predict(source="/content/gdrive/MyDrive/wound_management/data/test/", conf=0.25, save=True,
                        project="/content/gdrive/MyDrive/wound_management/output/output_n-seg/",
                        name="predict_wound" )


image 1/200 /content/gdrive/MyDrive/wound_management/data/test/1011.png: 640x640 1 wound, 12.9ms
image 2/200 /content/gdrive/MyDrive/wound_management/data/test/1014.png: 640x640 2 wounds, 11.6ms
image 3/200 /content/gdrive/MyDrive/wound_management/data/test/1016.png: 640x640 (no detections), 12.1ms
image 4/200 /content/gdrive/MyDrive/wound_management/data/test/1018.png: 640x640 1 wound, 10.5ms
image 5/200 /content/gdrive/MyDrive/wound_management/data/test/1020.png: 640x640 3 wounds, 11.6ms
image 6/200 /content/gdrive/MyDrive/wound_management/data/test/1021.png: 640x640 1 wound, 11.0ms
image 7/200 /content/gdrive/MyDrive/wound_management/data/test/1022.png: 640x640 1 wound, 13.0ms
image 8/200 /content/gdrive/MyDrive/wound_management/data/test/1024.png: 640x640 1 wound, 11.1ms
image 9/200 /content/gdrive/MyDrive/wound_management/data/test/1028.png: 640x640 1 wound, 10.4ms
image 10/200 /content/gdrive/MyDrive/wound_management/data/test/1029.png: 640x640 1 wound, 10.3ms
image 11/200 /cont

## B - yolov11n-seg for detecting wound (more data)

In [ ]:
"""Load a pretrained YOLO11n model"""

model1 = YOLO("yolo11n-seg.pt")

100%|██████████| 5.90M/5.90M [00:00<00:00, 382MB/s]


In [ ]:
"""Train the model"""

model1.train( data="/content/gdrive/MyDrive/wound_management/data1.yaml", epochs=250, batch=-1, patience=10)

Ultralytics 8.3.150 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/gdrive/MyDrive/wound_management/data1.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspectiv

100%|██████████| 755k/755k [00:00<00:00, 149MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

 12             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 13                  -1  1    111296  ultralytics.nn.modules.block.C3k2            [384, 128, 1, False]          
 14                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 15             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 16                  -1  1     32096  ultralytics.nn.modules.block.C3k2            [256, 64, 1, False]           
 17                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
 18            [-1, 13]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 19                  -1  1     86720  ultralytics.nn.modules.block.C3k2            [192, 128, 1, False]          
 20                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128,

100%|██████████| 5.35M/5.35M [00:00<00:00, 353MB/s]


...

                   all       1504        848      0.938      0.929      0.968      0.788      0.938      0.929      0.968      0.752
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 129, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



139 epochs completed in 2.481 hours.
Optimizer stripped from runs/segment/train/weights/last.pt, 6.0MB
Optimizer stripped from runs/segment/train/weights/best.pt, 6.0MB

Validating runs/segment/train/weights/best.pt...
Ultralytics 8.3.150 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11n-seg summary (fused): 113 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:06<00:00,  7.11it/s]


                   all       1504        848      0.934      0.925      0.969      0.789      0.934      0.925      0.967      0.753
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/segment/train


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7abcf877bf50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 

In [ ]:
"""Record the model"""

detect_path = Path("runs/segment")

# Find the actual train version
train_dirs = sorted(detect_path.glob("train*"), key=os.path.getmtime)

if not train_dirs:
    print(" Aucun dossier 'train*' trouvé dans runs/segment.")
else:
    latest_train_dir = train_dirs[-1]
    best_pt_path = latest_train_dir / "weights" / "best.pt"

    if best_pt_path.exists():
        # Copy it in google drive
        dest_path = "/content/gdrive/MyDrive/wound_management/wound_model_yolov11n-seg.pt"
        shutil.copy(best_pt_path, dest_path)
        print(f"Model copy from : {best_pt_path} in : {dest_path}")
    else:
        print(f"file not found {latest_train_dir}")

Model copy from : runs/segment/train/weights/best.pt in : /content/gdrive/MyDrive/wound_management/wound_model_yolov11n-seg.pt


In [ ]:
"""Validate the model"""

wound_model = YOLO("/content/gdrive/MyDrive/wound_management/wound_model_yolov11n-seg.pt")
wound_model = wound_model.val(data="/content/gdrive/MyDrive/wound_management/data1.yaml")

print("mAP50:", wound_model.seg.map50)
print("mAP50-95:", wound_model.seg.map)
print("Precision:", wound_model.seg.p)
print("Recall:", wound_model.seg.r)

Ultralytics 8.3.150 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11n-seg summary (fused): 113 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs
val: Fast image access ✅ (ping: 0.2±0.0 ms, read: 76.6±71.2 MB/s, size: 148.0 KB)


val: Scanning /content/gdrive/MyDrive/wound_management/data_buff/val/labels.cache... 752 images, 828 backgrounds, 0 corrupt: 100%|██████████| 1504/1504 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 94/94 [00:09<00:00,  9.59it/s]


                   all       1504        848      0.934      0.925      0.969      0.789      0.934      0.925      0.967      0.749
Speed: 0.4ms preprocess, 1.3ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/segment/val
mAP50: 0.9671139493865016
mAP50-95: 0.7487584173257112
Precision: [    0.93393]
Recall: [    0.92453]


In [ ]:
"""Made a prediction with the model"""

wound_model = YOLO("/content/gdrive/MyDrive/wound_management/wound_model_yolov11n-seg.pt")
# Verify the model is usable
results = wound_model.predict(source="/content/gdrive/MyDrive/wound_management/data_buff/test", conf=0.75)

total_images = len(results)
no_detection_count = 0

for r in results:
    if r.boxes is None or len(r.boxes) == 0:
        no_detection_count += 1
        continue

    for box in r.boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        class_name = wound_model.names[cls_id]

print(f"pourcentage of no detection at 75% : {no_detection_count / total_images:.2%}")


image 1/200 /content/gdrive/MyDrive/wound_management/data_buff/test/1011.png: 640x640 1 wound, 12.4ms
image 2/200 /content/gdrive/MyDrive/wound_management/data_buff/test/1014.png: 640x640 1 wound, 12.4ms
image 3/200 /content/gdrive/MyDrive/wound_management/data_buff/test/1016.png: 640x640 (no detections), 11.8ms
image 4/200 /content/gdrive/MyDrive/wound_management/data_buff/test/1018.png: 640x640 1 wound, 12.1ms
image 5/200 /content/gdrive/MyDrive/wound_management/data_buff/test/1020.png: 640x640 (no detections), 11.1ms
image 6/200 /content/gdrive/MyDrive/wound_management/data_buff/test/1021.png: 640x640 (no detections), 11.1ms
image 7/200 /content/gdrive/MyDrive/wound_management/data_buff/test/1022.png: 640x640 (no detections), 10.8ms
image 8/200 /content/gdrive/MyDrive/wound_management/data_buff/test/1024.png: 640x640 1 wound, 11.9ms
image 9/200 /content/gdrive/MyDrive/wound_management/data_buff/test/1028.png: 640x640 (no detections), 11.8ms
image 10/200 /content/gdrive/MyDrive/woun

KeyboardInterrupt: 

In [ ]:
"""Make a prediction and save the results"""

results = wound_model.predict(source="/content/gdrive/MyDrive/wound_management/data_buff/test/", conf=0.25, save=True,
                        project="/content/gdrive/MyDrive/wound_management/output/output_n-seg/",
                        name="predict_wound_buff" )


image 1/200 /content/gdrive/MyDrive/wound_management/data_buff/test/1011.png: 640x640 1 wound, 15.0ms
image 2/200 /content/gdrive/MyDrive/wound_management/data_buff/test/1014.png: 640x640 3 wounds, 12.2ms
image 3/200 /content/gdrive/MyDrive/wound_management/data_buff/test/1016.png: 640x640 1 wound, 13.7ms
image 4/200 /content/gdrive/MyDrive/wound_management/data_buff/test/1018.png: 640x640 1 wound, 13.3ms
image 5/200 /content/gdrive/MyDrive/wound_management/data_buff/test/1020.png: 640x640 1 wound, 12.9ms
image 6/200 /content/gdrive/MyDrive/wound_management/data_buff/test/1021.png: 640x640 1 wound, 13.2ms
image 7/200 /content/gdrive/MyDrive/wound_management/data_buff/test/1022.png: 640x640 (no detections), 13.1ms
image 8/200 /content/gdrive/MyDrive/wound_management/data_buff/test/1024.png: 640x640 1 wound, 13.7ms
image 9/200 /content/gdrive/MyDrive/wound_management/data_buff/test/1028.png: 640x640 1 wound, 19.1ms
image 10/200 /content/gdrive/MyDrive/wound_management/data_buff/test/102

## C - yolov11n-seg for detecting wound and the reference

In [ ]:
"""Load the model"""

model2 = YOLO("/content/gdrive/MyDrive/wound_management/wound_model_yolov11n-seg.pt")

In [ ]:
"""Train the model"""

model2.train( data="/content/gdrive/MyDrive/wound_management/data2.yaml", epochs=250, batch=-1, patience=10)

Ultralytics 8.3.150 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/gdrive/MyDrive/wound_management/data2.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/gdrive/MyDrive/wound_management/wound_model_yolov11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train42, nbs=64, nms=False, opset=None, optimize=False, op

train: Scanning /content/gdrive/MyDrive/wound_management/data_with_ref/train/labels.cache... 182 images, 182 backgrounds, 0 corrupt: 100%|██████████| 364/364 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA A100-SXM4-40GB) 39.56G total, 1.48G reserved, 0.29G allocated, 37.79G free


      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     2842998       10.36         1.017         34.76           nan        (1, 3, 640, 640)                    list
     2842998       20.71         1.409         39.23           nan        (2, 3, 640, 640)                    list
     2842998       41.43         2.248         36.95           nan        (4, 3, 640, 640)                    list
     2842998       82.85         3.714         42.07           nan        (8, 3, 640, 640)                    list
     2842998       165.7         6.780         44.74           nan       (16, 3, 640, 640)                    list
     2842998       331.4        12.646         49.43           nan       (32, 3, 640, 640)                    list
     2842998       662.8        24.176         64.91           nan       (64, 3, 640, 640)                    list
AutoBatch: Using batch-size 60 for CUDA:0 24.58G/39.56G (62%) ✅
train: Fast imag

train: Scanning /content/gdrive/MyDrive/wound_management/data_with_ref/train/labels.cache... 182 images, 182 backgrounds, 0 corrupt: 100%|██████████| 364/364 [00:00<?, ?it/s]

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.52it/s]


                   all         88         86      0.886      0.836      0.862      0.698      0.896      0.844      0.876      0.703
             reference         27         28      0.971      0.964       0.97      0.923      0.971      0.964       0.97      0.939
                 wound         42         58      0.802      0.707      0.755      0.473      0.821      0.724      0.782      0.467
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/segment/train42


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x787ab01955d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.04104

In [ ]:
"""Record the model"""

detect_path = Path("runs/segment")

# Find the actual train version
train_dirs = sorted(detect_path.glob("train*"), key=os.path.getmtime)

if not train_dirs:
    print(" Aucun dossier 'train*' trouvé dans runs/segment.")
else:
    latest_train_dir = train_dirs[-1]
    best_pt_path = latest_train_dir / "weights" / "best.pt"

    if best_pt_path.exists():
        # Copy it in google drive
        dest_path = "/content/gdrive/MyDrive/wound_management/wound_management_yolov11n-seg.pt"
        shutil.copy(best_pt_path, dest_path)
        print(f"Model copy from : {best_pt_path} in : {dest_path}")
    else:
        print(f"file not found {latest_train_dir}")

Model copy from : runs/segment/train42/weights/best.pt in : /content/gdrive/MyDrive/wound_management/wound_management_yolov11n-seg.pt


In [ ]:
"""Validate the model"""

wound_management = YOLO("/content/gdrive/MyDrive/wound_management/wound_management_yolov11n-seg.pt")
wound_management = wound_management.val(data="/content/gdrive/MyDrive/wound_management/data2.yaml")

print("mAP50:", wound_management.seg.map50)
print("mAP50-95:", wound_management.seg.map)
print("Precision:", wound_management.seg.p)
print("Recall:", wound_management.seg.r)

Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLO11n-seg summary (fused): 113 layers, 2,834,958 parameters, 0 gradients, 10.2 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 22.3MB/s]


val: Fast image access ✅ (ping: 0.7±0.2 ms, read: 0.0±0.0 MB/s, size: 19.7 KB)


val: Scanning /content/gdrive/MyDrive/wound_management/data_with_ref/val/labels... 44 images, 45 backgrounds, 0 corrupt: 100%|██████████| 88/88 [00:56<00:00,  1.55it/s]

val: New cache created: /content/gdrive/MyDrive/wound_management/data_with_ref/val/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:46<00:00,  7.75s/it]


                   all         88         86      0.887      0.837      0.863      0.699      0.897      0.845      0.876        0.7
             reference         27         28       0.97      0.964       0.97      0.924       0.97      0.964       0.97      0.939
                 wound         42         58      0.804       0.71      0.755      0.475      0.824      0.726      0.782      0.461
Speed: 9.5ms preprocess, 408.4ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to runs/segment/val
mAP50: 0.87591280028172
mAP50-95: 0.7004352409333497
Precision: [    0.97047     0.82394]
Recall: [    0.96429     0.72622]


In [ ]:
"""Made a prediction with the model"""

wound_management = YOLO("/content/gdrive/MyDrive/wound_management/basic_wound_management_yolov11n-seg.pt")

results = wound_management.predict(source="/content/gdrive/MyDrive/wound_management/data_with_ref/test/", conf=0.75)
total_images = len(results)
missing_class_0 = 0
missing_class_1 = 0
missing_both = 0

for r in results:
    if not r.boxes or len(r.boxes.cls) == 0:
        missing_both += 1
        missing_class_0 += 1
        missing_class_1 += 1
        continue

    detected_classes = [int(cls_id) for cls_id in r.boxes.cls]

    has_class_0 = 0 in detected_classes
    has_class_1 = 1 in detected_classes

    if not has_class_0:
        missing_class_0 += 1
    if not has_class_1:
        missing_class_1 += 1
    if not has_class_0 and not has_class_1:
        missing_both += 1

print(f"Images without reference : {missing_class_0} ({missing_class_0 / total_images:.2%})")
print(f"Images without wound : {missing_class_1} ({missing_class_1 / total_images:.2%})")
print(f"Nothing found : {missing_both} ({missing_both / total_images:.2%})")


image 1/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0017.png: 640x640 1 reference, 275.3ms
image 2/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0084.png: 640x640 1 reference, 344.3ms
image 3/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0110.png: 640x640 1 reference, 279.0ms
image 4/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0135.png: 640x640 1 reference, 1 wound, 292.3ms
image 5/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0634.png: 640x640 1 reference, 307.7ms
image 6/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0911.png: 640x640 1 reference, 271.1ms
image 7/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0935.png: 640x640 1 reference, 260.7ms
image 8/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/ADANIMG700.jpg: 640x640 1 reference, 274.3ms
image 9/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/ALFONSOIMG522.jpg: 640x6

In [ ]:
"""Make a prediction and save the results"""

results = wound_management.predict(source="/content/gdrive/MyDrive/wound_management/data_with_ref/test/", conf=0.25, save=True,
                        project="/content/gdrive/MyDrive/wound_management/output/output_n-seg/",
                        name="predict_both_buff" )


image 1/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0017.png: 640x640 1 reference, 1 wound, 277.6ms
image 2/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0084.png: 640x640 1 reference, 1 wound, 328.7ms
image 3/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0110.png: 640x640 2 references, 396.5ms
image 4/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0135.png: 640x640 1 reference, 1 wound, 402.7ms
image 5/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0634.png: 640x640 1 reference, 1 wound, 398.0ms
image 6/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0911.png: 640x640 1 reference, 2 wounds, 405.9ms
image 7/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0935.png: 640x640 2 references, 395.7ms
image 8/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/ADANIMG700.jpg: 640x640 1 reference, 2 wounds, 393.3ms
image 9/100 /content/gdrive/MyDrive/wound_manag

# II - yolov11x-seg


## A - yolov11x-seg for detecting wound (basic)

In [ ]:
"""Load a pretrained YOLO11n model"""

model = YOLO("yolo11x-seg.pt")

100%|██████████| 119M/119M [00:03<00:00, 35.0MB/s]


In [ ]:
"""Train the model"""

model.train( data="/content/gdrive/MyDrive/wound_management/data.yaml", epochs=250, batch=-1, patience=10)

Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/gdrive/MyDrive/wound_management/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11x-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspecti

train: Scanning /content/gdrive/MyDrive/wound_management/data/train/labels.cache... 809 images, 68 backgrounds, 0 corrupt: 100%|██████████| 809/809 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA A100-SXM4-40GB) 39.56G total, 13.24G reserved, 2.37G allocated, 23.95G free


      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    62051411       319.7        12.822         66.62           nan        (1, 3, 640, 640)                    list
    62051411       639.4        12.822         63.99           nan        (2, 3, 640, 640)                    list
    62051411        1279        15.632         68.35           nan        (4, 3, 640, 640)                    list
    62051411        2557        26.758         84.57           nan        (8, 3, 640, 640)                    list
    62051411        5115        46.815         100.5           nan       (16, 3, 640, 640)                    list
CUDA out of memory. Tried to allocate 300.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 156.88 MiB is free. Process 11768 has 39.39 GiB memory in use. Of the allocated memory 38.49 GiB is allocated by PyTorch, and 350.20 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory

train: Scanning /content/gdrive/MyDrive/wound_management/data/train/labels.cache... 809 images, 68 backgrounds, 0 corrupt: 100%|██████████| 809/809 [00:00<?, ?it/s]

...

                   all        200        221      0.894      0.855      0.923      0.645      0.905       0.86      0.922      0.647
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 28, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



38 epochs completed in 0.747 hours.
Optimizer stripped from runs/segment/train2/weights/last.pt, 124.8MB
Optimizer stripped from runs/segment/train2/weights/best.pt, 124.8MB

Validating runs/segment/train2/weights/best.pt...
Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11x-seg summary (fused): 203 layers, 62,003,283 parameters, 0 gradients, 318.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:02<00:00, 18.56it/s]


                   all        200        221      0.903      0.842      0.929      0.666      0.891      0.833      0.919      0.661
Speed: 0.3ms preprocess, 6.9ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/segment/train2


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b614d2dfc90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 

In [ ]:
"""Record the model"""

detect_path = Path("runs/segment")

# Find the actual train version
train_dirs = sorted(detect_path.glob("train*"), key=os.path.getmtime)

if not train_dirs:
    print(" Aucun dossier 'train*' trouvé dans runs/segment.")
else:
    latest_train_dir = train_dirs[-1]
    best_pt_path = latest_train_dir / "weights" / "best.pt"

    if best_pt_path.exists():
        # Copy it in google drive
        dest_path = "/content/gdrive/MyDrive/wound_management/basic_wound_model_yolov11x-seg.pt"
        shutil.copy(best_pt_path, dest_path)
        print(f"Model copy from : {best_pt_path} in : {dest_path}")
    else:
        print(f"file not found {latest_train_dir}")

Model copy from : runs/segment/train2/weights/best.pt in : /content/gdrive/MyDrive/wound_management/basic_wound_model_yolov11x-seg.pt


In [ ]:
"""Validate the model"""

wound_model = YOLO("/content/gdrive/MyDrive/wound_management/basic_wound_model_yolov11x-seg.pt")
wound_model = wound_model.val(data="/content/gdrive/MyDrive/wound_management/data.yaml")

print("mAP50:", wound_model.seg.map50)
print("mAP50-95:", wound_model.seg.map)
print("Precision:", wound_model.seg.p)
print("Recall:", wound_model.seg.r)

Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11x-seg summary (fused): 203 layers, 62,003,283 parameters, 0 gradients, 318.5 GFLOPs
val: Fast image access ✅ (ping: 0.3±0.1 ms, read: 155.3±60.4 MB/s, size: 251.5 KB)


val: Scanning /content/gdrive/MyDrive/wound_management/data/val/labels.cache... 200 images, 22 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:03<00:00,  3.40it/s]


                   all        200        221      0.904      0.848      0.929      0.667      0.894       0.84       0.92      0.661
Speed: 1.2ms preprocess, 7.2ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/segment/val2
mAP50: 0.9195483003236898
mAP50-95: 0.6611052115537537
Precision: [    0.89399]
Recall: [    0.83951]


In [ ]:
"""Made a prediction with the model"""

wound_model = YOLO("/content/gdrive/MyDrive/wound_management/basic_wound_model_yolov11x-seg.pt")

results = wound_model.predict(source="/content/gdrive/MyDrive/wound_management/data/test", conf=0.75)

total_images = len(results)
no_detection_count = 0

for r in results:
    if r.boxes is None or len(r.boxes) == 0:
        no_detection_count += 1
        continue

    for box in r.boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        class_name = wound_model.names[cls_id]

print(f"pourcentage of no detection at 75% : {no_detection_count / total_images:.2%}")


image 1/200 /content/gdrive/MyDrive/wound_management/data/test/1011.png: 640x640 1 wound, 22.1ms
image 2/200 /content/gdrive/MyDrive/wound_management/data/test/1014.png: 640x640 1 wound, 20.3ms
image 3/200 /content/gdrive/MyDrive/wound_management/data/test/1016.png: 640x640 (no detections), 27.0ms
image 4/200 /content/gdrive/MyDrive/wound_management/data/test/1018.png: 640x640 1 wound, 22.1ms
image 5/200 /content/gdrive/MyDrive/wound_management/data/test/1020.png: 640x640 (no detections), 21.0ms
image 6/200 /content/gdrive/MyDrive/wound_management/data/test/1021.png: 640x640 (no detections), 21.5ms
image 7/200 /content/gdrive/MyDrive/wound_management/data/test/1022.png: 640x640 (no detections), 21.3ms
image 8/200 /content/gdrive/MyDrive/wound_management/data/test/1024.png: 640x640 (no detections), 21.3ms
image 9/200 /content/gdrive/MyDrive/wound_management/data/test/1028.png: 640x640 (no detections), 21.3ms
image 10/200 /content/gdrive/MyDrive/wound_management/data/test/1029.png: 640x

In [ ]:
"""Make a prediction and save the results"""

results = wound_model.predict(source="/content/gdrive/MyDrive/wound_management/data/test/", conf=0.25, save=True,
                        project="/content/gdrive/MyDrive/wound_management/output/output_x-seg/",
                        name="predict_wound" )


image 1/200 /content/gdrive/MyDrive/wound_management/data/test/1011.png: 640x640 1 wound, 24.5ms
image 2/200 /content/gdrive/MyDrive/wound_management/data/test/1014.png: 640x640 2 wounds, 22.3ms
image 3/200 /content/gdrive/MyDrive/wound_management/data/test/1016.png: 640x640 1 wound, 25.3ms
image 4/200 /content/gdrive/MyDrive/wound_management/data/test/1018.png: 640x640 1 wound, 22.7ms
image 5/200 /content/gdrive/MyDrive/wound_management/data/test/1020.png: 640x640 (no detections), 23.5ms
image 6/200 /content/gdrive/MyDrive/wound_management/data/test/1021.png: 640x640 1 wound, 21.9ms
image 7/200 /content/gdrive/MyDrive/wound_management/data/test/1022.png: 640x640 1 wound, 22.3ms
image 8/200 /content/gdrive/MyDrive/wound_management/data/test/1024.png: 640x640 1 wound, 21.9ms
image 9/200 /content/gdrive/MyDrive/wound_management/data/test/1028.png: 640x640 1 wound, 22.1ms
image 10/200 /content/gdrive/MyDrive/wound_management/data/test/1029.png: 640x640 1 wound, 21.8ms
image 11/200 /conte

## B - yolov11n-seg for detecting wound (more data)

In [ ]:
"""Load a pretrained YOLO11n model"""

model1 = YOLO("yolo11x-seg.pt")

100%|██████████| 119M/119M [00:00<00:00, 296MB/s]


In [ ]:
"""Train the model"""

model1.train( data="/content/gdrive/MyDrive/wound_management/data1.yaml", epochs=250, batch=-1, patience=10)

Ultralytics 8.3.151 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/gdrive/MyDrive/wound_management/data1.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11x-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspectiv

100%|██████████| 755k/755k [00:00<00:00, 14.2MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2784  ultralytics.nn.modules.conv.Conv             [3, 96, 3, 2]                 
  1                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  2                  -1  2    389760  ultralytics.nn.modules.block.C3k2            [192, 384, 2, True, 0.25]     
  3                  -1  1   1327872  ultralytics.nn.modules.conv.Conv             [384, 384, 3, 2]              
  4                  -1  2   1553664  ultralytics.nn.modules.block.C3k2            [384, 768, 2, True, 0.25]     
  5                  -1  1   5309952  ultralytics.nn.modules.conv.Conv             [768, 768, 3, 2]              


  6                  -1  2   5022720  ultralytics.nn.modules.block.C3k2            [768, 768, 2, True]           
  7                  -1  1   5309952  ultralytics.nn.modules.conv.Conv             [768, 768, 3, 2]              
  8                  -1  2   5022720  ultralytics.nn.modules.block.C3k2            [768, 768, 2, True]           
  9                  -1  1   1476864  ultralytics.nn.modules.block.SPPF            [768, 768, 5]                 
 10                  -1  2   3264768  ultralytics.nn.modules.block.C2PSA           [768, 768, 2]                 
 11                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 12             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 13                  -1  2   5612544  ultralytics.nn.modules.block.C3k2            [1536, 768, 2, True]          
 14                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None

100%|██████████| 5.35M/5.35M [00:00<00:00, 66.5MB/s]


...


      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


    113/250      13.5G     0.8095      1.213     0.5564      1.062         13        640: 100%|██████████| 862/862 [02:40<00:00,  5.38it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 108/108 [00:13<00:00,  8.27it/s]

                   all       1504        848      0.939      0.956      0.979      0.808      0.957      0.938      0.974      0.759



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


    114/250      13.6G     0.7942      1.157     0.5604      1.061          9        640:  74%|███████▍  | 640/862 [01:58<00:41,  5.33it/s]

In [ ]:
"""Record the model"""

detect_path = Path("runs/segment")

# Find the actual train version
train_dirs = sorted(detect_path.glob("train*"), key=os.path.getmtime)

if not train_dirs:
    print(" Aucun dossier 'train*' trouvé dans runs/segment.")
else:
    latest_train_dir = train_dirs[-1]
    best_pt_path = latest_train_dir / "weights" / "best.pt"

    if best_pt_path.exists():
        # Copy it in google drive
        dest_path = "/content/gdrive/MyDrive/wound_management/wound_model_yolov11x-seg.pt"
        shutil.copy(best_pt_path, dest_path)
        print(f"Model copy from : {best_pt_path} in : {dest_path}")
    else:
        print(f"file not found {latest_train_dir}")

In [ ]:
""" Validate the model"""

wound_model_buff = YOLO("/content/gdrive/MyDrive/wound_management/wound_model_yolov11x-seg.pt")
wound_model_buff = wound_model_buff.val(data="/content/gdrive/MyDrive/wound_management/data1.yaml")

print("mAP50:", wound_model_buff.seg.map50)
print("mAP50-95:", wound_model_buff.seg.map)
print("Precision:", wound_model_buff.seg.p)
print("Recall:", wound_model_buff.seg.r)

In [ ]:
"""Made a prediction with the model"""

wound_model_buff = YOLO("/content/gdrive/MyDrive/wound_management/wound_model_yolov11x-seg.pt")
# Verify the model is usable
results = wound_model_buff.predict(source="/content/gdrive/MyDrive/wound_management/data1/test", conf=0.75)

total_images = len(results)
no_detection_count = 0

for r in results:
    if r.boxes is None or len(r.boxes) == 0:
        no_detection_count += 1
        continue

    for box in r.boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        class_name = wound_model_buff.names[cls_id]

print(f"pourcentage of no detection at 75% : {no_detection_count / total_images:.2%}")

In [ ]:
"""Make a prediction and save the results"""
results = wound_model_buff.predict(source="/content/gdrive/MyDrive/wound_management/data1/test/", conf=0.25, save=True,
                        project="/content/gdrive/MyDrive/wound_management/output/output_x-seg/",
                        name="predict_wound_buff" )

## C - yolov11x-seg for detecting wound and the reference

In [ ]:
"""Load the model"""

model2 = YOLO("/content/gdrive/MyDrive/wound_management/basic_wound_model_yolov11x-seg.pt")

In [ ]:
"""Train the model"""

model2.train( data="/content/gdrive/MyDrive/wound_management/data2.yaml", epochs=250, batch=-1, patience=10)

Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/gdrive/MyDrive/wound_management/data2.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/gdrive/MyDrive/wound_management/basic_wound_model_yolov11x-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=

train: Scanning /content/gdrive/MyDrive/wound_management/data_with_ref/train/labels.cache... 182 images, 0 backgrounds, 0 corrupt: 100%|██████████| 182/182 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (Tesla T4) 14.74G total, 2.86G reserved, 0.79G allocated, 11.09G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output


    62052566       319.7         5.557         69.01           nan        (1, 3, 640, 640)                    list
    62052566       639.4         9.232         93.81           nan        (2, 3, 640, 640)                    list
    62052566        1279        14.298         125.6           nan        (4, 3, 640, 640)                    list
    62052566        2557        25.485         255.6           nan        (8, 3, 640, 640)                    list
CUDA out of memory. Tried to allocate 76.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 14.12 MiB is free. Process 22322 has 14.72 GiB memory in use. Of the allocated memory 13.92 GiB is allocated by PyTorch, and 630.38 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
AutoBatch: Using batch-

train: Scanning /content/gdrive/MyDrive/wound_management/data_with_ref/train/labels.cache... 182 images, 0 backgrounds, 0 corrupt: 100%|██████████| 182/182 [00:00<?, ?it/s]

...

                   all         44         86       0.92      0.741       0.81       0.64      0.934      0.749      0.818      0.653


EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 49, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

59 epochs completed in 1.093 hours.
Optimizer stripped from runs/segment/train3/weights/last.pt, 124.8MB
Optimizer stripped from runs/segment/train3/weights/best.pt, 124.8MB

Validating runs/segment/train3/weights/best.pt...
Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11x-seg summary (fused): 203 layers, 62,004,438 parameters, 0 gradients, 318.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:02<00:00,  9.74it/s]


                   all         44         86      0.856      0.792      0.826      0.669      0.856      0.792       0.83      0.669
             reference         27         28      0.984      0.964      0.968      0.937      0.984      0.964      0.968      0.935
                 wound         42         58      0.728      0.621      0.684        0.4      0.728      0.621      0.692      0.403
Speed: 0.5ms preprocess, 38.3ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/segment/train3


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7bc2bc3e8710>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.04104

In [ ]:
"""Record the model"""

detect_path = Path("runs/segment")

# Find the actual train version
train_dirs = sorted(detect_path.glob("train*"), key=os.path.getmtime)

if not train_dirs:
    print(" Aucun dossier 'train*' trouvé dans runs/segment.")
else:
    latest_train_dir = train_dirs[-1]
    best_pt_path = latest_train_dir / "weights" / "best.pt"

    if best_pt_path.exists():
        # Copy it in google drive
        dest_path = "/content/gdrive/MyDrive/wound_management/basic_wound_management_yolov11x-seg.pt"
        shutil.copy(best_pt_path, dest_path)
        print(f"Model copy from : {best_pt_path} in : {dest_path}")
    else:
        print(f"file not found {latest_train_dir}")

Model copy from : runs/segment/train3/weights/best.pt in : /content/gdrive/MyDrive/wound_management/basic_wound_management_yolov11x-seg.pt


In [ ]:
"""Validate the model"""

wound_management = YOLO("/content/gdrive/MyDrive/wound_management/basic_wound_management_yolov11x-seg.pt")
wound_management = wound_management.val(data="/content/gdrive/MyDrive/wound_management/data2.yaml")

print("mAP50:", wound_management.seg.map50)
print("mAP50-95:", wound_management.seg.map)
print("Precision:", wound_management.seg.p)
print("Recall:", wound_management.seg.r)

Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11x-seg summary (fused): 203 layers, 62,004,438 parameters, 0 gradients, 318.5 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 131.2±149.1 MB/s, size: 421.6 KB)


val: Scanning /content/gdrive/MyDrive/wound_management/data_with_ref/val/labels.cache... 44 images, 1 backgrounds, 0 corrupt: 100%|██████████| 44/44 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  33%|███▎      | 1/3 [00:07<00:14,  7.48s/it]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  67%|██████▋   | 2/3 [00:09<00:04,  4.36s/it]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:11<00:00,  3.83s/it]


                   all         44         86      0.856      0.792      0.826      0.668      0.856      0.792      0.829      0.666
             reference         27         28      0.984      0.964      0.968      0.937      0.984      0.964      0.968      0.931
                 wound         42         58      0.727      0.621      0.684      0.399      0.727      0.621      0.691      0.401
Speed: 5.8ms preprocess, 73.8ms inference, 0.0ms loss, 7.2ms postprocess per image
Results saved to runs/segment/val3
mAP50: 0.829391855188164
mAP50-95: 0.6659507188635246
Precision: [     0.9841     0.72696]
Recall: [    0.96429     0.62069]


In [ ]:
"""Made a prediction with the model"""

wound_model = YOLO("/content/gdrive/MyDrive/wound_management/basic_wound_management_yolov11x-seg.pt")
# Verify the model is usable
results = wound_model.predict(source="/content/gdrive/MyDrive/wound_management/data_with_ref/test", conf=0.75)

total_images = len(results)
missing_class_0 = 0
missing_class_1 = 0
missing_both = 0

for r in results:
    if not r.boxes or len(r.boxes.cls) == 0:
        missing_both += 1
        missing_class_0 += 1
        missing_class_1 += 1
        continue

    detected_classes = [int(cls_id) for cls_id in r.boxes.cls]

    has_class_0 = 0 in detected_classes
    has_class_1 = 1 in detected_classes

    if not has_class_0:
        missing_class_0 += 1
    if not has_class_1:
        missing_class_1 += 1
    if not has_class_0 and not has_class_1:
        missing_both += 1

print(f"Images without reference : {missing_class_0} ({missing_class_0 / total_images:.2%})")
print(f"Images without wound: {missing_class_1} ({missing_class_1 / total_images:.2%})")
print(f"Nothing found : {missing_both} ({missing_both / total_images:.2%})")


image 1/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0017.png: 640x640 1 reference, 7249.4ms
image 2/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0084.png: 640x640 1 reference, 5308.5ms
image 3/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0110.png: 640x640 1 reference, 5583.6ms
image 4/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0135.png: 640x640 1 reference, 6304.2ms
image 5/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0634.png: 640x640 1 reference, 5327.0ms
image 6/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0911.png: 640x640 1 reference, 6811.9ms
image 7/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0935.png: 640x640 1 reference, 5238.0ms
image 8/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/ADANIMG700.jpg: 640x640 1 reference, 6262.6ms
image 9/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/ALFONSOIMG522.jpg: 640x64

In [ ]:
"""Make a prediction and save the results"""

results = wound_model.predict(source="/content/gdrive/MyDrive/wound_management/data_with_ref/test/", conf=0.25, save=True,
                        project="/content/gdrive/MyDrive/wound_management/output/output_x-seg/",
                        name="predict_both" )


image 1/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0017.png: 640x640 1 reference, 1 wound, 6862.9ms
image 2/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0084.png: 640x640 1 reference, 1 wound, 5213.5ms
image 3/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0110.png: 640x640 1 reference, 6423.1ms
image 4/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0135.png: 640x640 1 reference, 2 wounds, 5174.9ms
image 5/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0634.png: 640x640 1 reference, 1 wound, 5198.9ms
image 6/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0911.png: 640x640 1 reference, 6312.8ms
image 7/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/0935.png: 640x640 2 references, 1 wound, 5193.0ms
image 8/100 /content/gdrive/MyDrive/wound_management/data_with_ref/test/ADANIMG700.jpg: 640x640 1 reference, 2 wounds, 6585.8ms
image 9/100 /content/gdrive/MyDrive/woun

# III - resnet50

## A - Detecting wound

In [ ]:
""" Step 1 : Load the yaml """

with open("/content/gdrive/MyDrive/wound_management/data_pytorch.yaml") as f:
    data = yaml.safe_load(f)

train_images_dir = data['train']
val_images_dir = data['val']
train_masks_dir = data['train_masks']
val_masks_dir = data['val_masks']
num_classes = data['nc']
class_names = data['names']

In [ ]:
""" Step 2 : Create the data for the model """

class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform_img=None, transform_mask=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_list = os.listdir(image_dir)
        self.transform_img = transform_img
        self.transform_mask = transform_mask

    def __len__(self):
        return len(self.image_list)

    # Find the image and the mask for it (even if ot's in jpg or png)
    def __getitem__(self, idx):
        image_name = self.image_list[idx]
        base_name = os.path.splitext(image_name)[0]
        image_path = os.path.join(self.image_dir, image_name)

        mask_path = None
        for ext in ['.png', '.jpg']:
            potential_path = os.path.join(self.mask_dir, base_name + ext)
            if os.path.exists(potential_path):
                mask_path = potential_path
                break

        if mask_path is None:
            raise FileNotFoundError(f"Mask not found for image: {image_name}")

        image = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.transform_img:
            image = self.transform_img(image)
        if self.transform_mask:
            mask = self.transform_mask(mask)

        mask = mask.float() / 255.0

        return image, mask

""" Step 3 : Transforms """
transform_img = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
])

transform_mask = transforms.Compose([
    transforms.Resize((384, 384), interpolation=Image.NEAREST),
    transforms.PILToTensor(),
])

train_dataset = SegmentationDataset(train_images_dir, train_masks_dir, transform_img, transform_mask)
val_dataset = SegmentationDataset(val_images_dir, val_masks_dir, transform_img, transform_mask)

print(f"size of trainning : {len(train_dataset)}")
print(f"size of validation : {len(val_dataset)}")

# shuffle=True for training helps the model generalize and shuffle=False for validation ensures consistent evaluation.

train_loader = DataLoader(
    train_dataset,
    batch_size=32, # faible pour les test
    shuffle=True,
    num_workers=24,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32, # faible pour les test
    shuffle=False,
    num_workers=24,
    pin_memory=True
)

train_dataset = SegmentationDataset(train_images_dir, train_masks_dir, transform_img, transform_mask)

size of trainning : 182
size of validation : 44


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
""" Step 4 : Load the model, optimiser """

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device : cuda mean the GPU is connected
print(f"device : {device}")

model = seg_models.deeplabv3_resnet50(pretrained=True)
model.classifier[4] = nn.Conv2d(256, 1, kernel_size=1)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scaler = torch.cuda.amp.GradScaler()
torch.backends.cudnn.benchmark = True

device : cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 178, in close
    self._close()
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 377, in _close
Ex

In [ ]:
""" Step 5 : loss """

def dice_loss(pred, target, smooth=1e-6):
    pred = pred.view(-1)
    target = target.view(-1)
    intersection = (pred * target).sum()
    return 1 - ((2. * intersection + smooth) / (pred.sum() + target.sum() + smooth))

def combined_loss(logits, target):
    bce = F.binary_cross_entropy_with_logits(logits, target)
    probs = torch.sigmoid(logits)
    dice = dice_loss(probs, target)
    return bce + dice

In [ ]:
""" Step 6 : Training """

best_val_dice = 0.0
num_epochs = 10 # faible pour les test

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    print(f"\nEpoch {epoch+1}/{num_epochs}")
    for batch_idx, (images, masks) in enumerate(train_loader):
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()

        with torch.amp.autocast(device_type="cuda"):
            logits = model(images)['out']
            loss = combined_loss(logits, masks)

            scaler.scale(loss).backward()
            torch.cuda.empty_cache()
            scaler.step(optimizer)
            scaler.update()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1} - Average Loss: {avg_train_loss:.4f}")

    """ Step 8 : Validation """

    model.eval()
    val_loss = 0.0
    dice_total = 0.0
    with torch.no_grad():
        for images, masks in val_loader:
            images, masks = images.to(device), masks.to(device)
            outputs = torch.sigmoid(model(images)['out'])
            loss = combined_loss(outputs, masks)
            val_loss += loss.item()

            dice_batch = 1 - dice_loss(outputs, masks)
            dice_total += dice_batch.item()

    avg_val_loss = val_loss / len(val_loader)
    avg_dice = dice_total / len(val_loader)
    print(f"Validation - Loss: {avg_val_loss:.4f} | Dice: {avg_dice:.4f}")

    """ Step 7 : Saving the best model """
    if avg_dice > 0.9:
        save_path = "/content/gdrive/MyDrive/wound_management/wound_model_deeplabv3.pth"
        torch.save(model.state_dict(), save_path)
        print(f"Best model saved in: {save_path}")


Epoch 1/10


<ipython-input-36-1554646439>:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x795ed1ddef20>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1582, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.11/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 948, in wait


KeyboardInterrupt: 

## B - Detecting wound and the reference


In [ ]:
""" Step 1 : Load the yaml """

with open("/content/gdrive/MyDrive/wound_management/data_pytorch_1.yaml") as f:
    data = yaml.safe_load(f)

train_images_dir = data['train']
val_images_dir = data['val']
train_masks_dir = data['train_masks']
val_masks_dir = data['val_masks']
num_classes = data['nc']
class_names = data['names']

In [ ]:
""" Step 2 : Create the data for the model """

class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform_img=None, transform_mask=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_list = os.listdir(image_dir)
        self.transform_img = transform_img
        self.transform_mask = transform_mask

    def __len__(self):
        return len(self.image_list)

    # Find the image and the mask for it (even if ot's in jpg or png)
    def __getitem__(self, idx):
        image_name = self.image_list[idx]
        base_name = os.path.splitext(image_name)[0]
        image_path = os.path.join(self.image_dir, image_name)

        mask_path = None
        for ext in ['.png', '.jpg']:
            potential_path = os.path.join(self.mask_dir, base_name + ext)
            if os.path.exists(potential_path):
                mask_path = potential_path
                break

        if mask_path is None:
            raise FileNotFoundError(f"Mask not found for image: {image_name}")

        image = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.transform_img:
            image = self.transform_img(image)
        if self.transform_mask:
            mask = self.transform_mask(mask)

        mask = mask.float() / 255.0

        return image, mask

""" Step 3 : Transforms """
transform_img = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
])

transform_mask = transforms.Compose([
    transforms.Resize((384, 384), interpolation=Image.NEAREST),
    transforms.PILToTensor(),
])

train_dataset = SegmentationDataset(train_images_dir, train_masks_dir, transform_img, transform_mask)
val_dataset = SegmentationDataset(val_images_dir, val_masks_dir, transform_img, transform_mask)

print(f"size of trainning : {len(train_dataset)}")
print(f"size of validation : {len(val_dataset)}")

# shuffle=True for training helps the model generalize and shuffle=False for validation ensures consistent evaluation.

train_loader = DataLoader(
    train_dataset,
    batch_size=32, # faible pour les test
    shuffle=True,
    num_workers=8,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32, # faible pour les test
    shuffle=False,
    num_workers=8,
    pin_memory=True
)

size of trainning : 182
size of validation : 44


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
""" Step 4 : Load the model, optimiser """

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device : cuda mean the GPU is connected
print(f"device : {device}")

model = seg_models.deeplabv3_resnet50(pretrained=False, num_classes=2, aux_loss=True)
state_dict = torch.load("/content/gdrive/MyDrive/wound_management/wound_model_deeplabv3.pth", map_location='cpu')
filtered_state_dict = {k: v for k, v in state_dict.items() if not k.startswith('classifier.4') and not k.startswith('aux_classifier.4')}
model.load_state_dict(filtered_state_dict, strict=False)

model.classifier[4] = nn.Conv2d(256, 1, kernel_size=1)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scaler = torch.cuda.amp.GradScaler()
torch.backends.cudnn.benchmark = True

device : cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 205MB/s]
<ipython-input-11-108074041>:17: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [ ]:
""" Step 5 : loss """

def dice_loss(pred, target, smooth=1e-6):
    pred = pred.view(-1)
    target = target.view(-1)
    intersection = (pred * target).sum()
    return 1 - ((2. * intersection + smooth) / (pred.sum() + target.sum() + smooth))

def combined_loss(logits, target):
    bce = F.binary_cross_entropy_with_logits(logits, target)
    probs = torch.sigmoid(logits)
    dice = dice_loss(probs, target)
    return bce + dice

In [ ]:
""" Step 6 : Training """

best_val_dice = 0.0
num_epochs = 25
patience = 3
epochs_without_improvement = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    print(f"\nEpoch {epoch+1}/{num_epochs}")
    for batch_idx, (images, masks) in enumerate(train_loader):
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()

        with torch.amp.autocast(device_type="cuda"):
            logits = model(images)['out']
            loss = combined_loss(logits, masks)

            scaler.scale(loss).backward()
            torch.cuda.empty_cache()
            scaler.step(optimizer)
            scaler.update()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1} - Average Loss: {avg_train_loss:.4f}")

    """ Step 7 : Validation """

    model.eval()
    val_loss = 0.0
    dice_total = 0.0
    with torch.no_grad():
        for images, masks in val_loader:
            images, masks = images.to(device), masks.to(device)
            outputs = torch.sigmoid(model(images)['out'])
            loss = combined_loss(outputs, masks)
            val_loss += loss.item()

            dice_batch = 1 - dice_loss(outputs, masks)
            dice_total += dice_batch.item()

    avg_val_loss = val_loss / len(val_loader)
    avg_dice = dice_total / len(val_loader)
    print(f"Validation - Loss: {avg_val_loss:.4f} | Dice: {avg_dice:.4f}")

    """ Step 8 : Save model if Dice improves """

    if avg_dice > 0.8:
        best_val_dice = avg_dice
        epochs_without_improvement = 0
        save_path = "/content/gdrive/MyDrive/wound_management/wound_management_deeplabv3.pth"
        torch.save(model.state_dict(), save_path)
        print(f"Best model saved in: {save_path} (Dice: {avg_dice:.4f})")

    else:
        epochs_without_improvement += 1
        print(f"Dice did not improve. Patience counter: {epochs_without_improvement}/{patience}")

    """ Step 9 : Early stopping """

    if epochs_without_improvement >= patience:
        print(f"\n Early stopping triggered after {epoch+1} epochs.")
        break


Epoch 1/10
Epoch 1 - Average Loss: 1.4200
Validation - Loss: 1.8010 | Dice: 0.1450

Epoch 2/10
Epoch 2 - Average Loss: 1.3223
Validation - Loss: 1.7807 | Dice: 0.1714

Epoch 3/10
Epoch 3 - Average Loss: 1.2217
Validation - Loss: 1.7585 | Dice: 0.1882

Epoch 4/10
Epoch 4 - Average Loss: 1.1419
Validation - Loss: 1.7319 | Dice: 0.2176

Epoch 5/10
Epoch 5 - Average Loss: 1.0786
Validation - Loss: 1.7088 | Dice: 0.2484

Epoch 6/10
Epoch 6 - Average Loss: 1.0305
Validation - Loss: 1.6973 | Dice: 0.2658

Epoch 7/10
Epoch 7 - Average Loss: 0.9816
Validation - Loss: 1.6905 | Dice: 0.2741

Epoch 8/10
Epoch 8 - Average Loss: 0.9451
Validation - Loss: 1.6770 | Dice: 0.2951

Epoch 9/10
Epoch 9 - Average Loss: 0.9191
Validation - Loss: 1.6684 | Dice: 0.3116

Epoch 10/10
Epoch 10 - Average Loss: 0.8787
Validation - Loss: 1.6594 | Dice: 0.3303


# IV - resnet101

# V - Estimate the healing time

## A - Load, segment and call the imput


In [9]:
def load_image(path_img):
    """Load an image from the specified path."""

    img = cv2.imread(path_img)
    if img is None:
        raise FileNotFoundError(f"Image at {path_img} not found.")
    H, W, _ = img.shape
    return img, H, W

def load_model_yolo(path_model):
    """Load the YOLO model from the specified path."""

    return YOLO(path_model)

def load_model_resnet(path_model):
    """Load the resnet model from the specified path."""
    model = seg_models.deeplabv3_resnet50(pretrained=False,num_classes=2, aux_loss=True)
    model.load_state_dict(torch.load(path_model, map_location=torch.device('cpu')))
    model.eval()
    return model

def segment_image_yolo(model, img):
    """Segment the image using the YOLO model."""

    results = model.predict([img], conf=0.25)
    return results

def segment_image_resnet(model, img):
    """Segment the image using the resnet model."""
    model.to(device)

    transform = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ])

    input_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        results = model(input_tensor)

    return results.squeeze(0)

def imput_parameter(image_file):
  """Call of the input at one time"""

  age = int(input(f"How old is the patient ({image_file})? "))

  wound_type = input(f"What is the wound type ({image_file})? (e.g., skintear, burn, diabetic foot): ").strip().lower()
  localisation = input(f"Where is the wound ({image_file})? (e.g., feet, heels, legs): ").strip().lower()
  comorbidities = input(f"Any other conditions ({image_file})? (e.g., diabete, smoking, hiv, can be more than one): ").strip().lower()

  return age, wound_type, localisation, comorbidities

## B - Parameter list and validation of the imput

In [10]:
# healing speed and standard deviation for a particular type of wound
HEALING_SPEED_TABLE = {
      'atheroma': (0.44, 0.39),
      'automutilation': (0.11, 0.00),
      'bite and scratch': (0.64, 0.72),
      'blister': (0.54, 0.55),
      'burn': (3.37, 9.98),
      'cellulitis': (0.75, 1.16),
      'clavus': (0.10, 0.03),
      'decubitus': (0.30, 0.34),
      'dermatological': (0.52, 0.49),
      'diabetic foot': (0.05, 0.03),
      'edema': (2.37, 5.28),
      'erysipelas': (3.31, 9.99),
      'gout': (0.18, 0.28),
      'hidradenitis': (0.26, 0.22),
      'oncological': (0.26, 0.19),
      'other': (0.61, 1.45),
      'paronychia': (0.19, 0.22),
      'postoperative': (0.64, 1.33),
      'sinus pilonidales': (0.14, 0.00),
      'skintear': (0.50, 0.78),
      'trauma': (0.61, 1.08),
      'ulcus cruris': (0.25, 0.34),
      'ulcus cruris arteriosum': (0.13, 0.15),
      'ulcus cruris venosum': (0.59, 1.13),
      'unguius incarnatus': (0.31, 0.35)
}

# factor of healing depending on the location
LOCALISATION_COEFFICIENTS = {
      'face': 1.2,
      'arms': 0.9,
      'hands': 0.9,
      'back': 1.0,
      'abdomen': 1.0,
      'legs': 0.75,
      'feet': 0.6,
      'heels': 0.5,
      'sacrum': 0.6,
      'groin': 0.7,
      'other': 1.0
}

# minimum factor of healing by age category
age_coef_min = {
      (0, 10): 1.1,
      (11, 25): 1.0,
      (26, 40): 0.9,
      (41, 60): 0.75,
      (61, 75): 0.5,
      (76, 90): 0.3,
      (91, 120): 0.2
}

# maximum factor of healing by age category
age_coef_max = {
      (0, 10): 1.2,
      (11, 25): 1.0,
      (26, 40): 1.0,
      (41, 60): 0.9,
      (61, 75): 0.75,
      (76, 90): 0.5,
      (91, 120): 0.3
}

# factor of healing for comorbidities
known_comorbidities = {
      'diabete': 0.8,
      'smoking': 0.9,
      'hiv': 0.6,
      'malnutrition': 0.7,
      'hypertension' : 0.9,
      'obesity' : 0.85,
      'alcoholism' : 0.85,
      'anemia' : 0.85,
      'neuropathy' : 0.75,
      'cancer' : 0.7,
      'kidney failure' : 0.8,
      'immune system deficiency' : 0.85,
      'none' : 1.0
}

# factor of healing depending on the infection state
INFECTION_LEVELS = {
      'none': 1.0,
      'low': 0.85,
      'medium': 0.65,
      'high': 0.4
}

# factor of healing depending on the necrosis state
NECROSIS_LEVELS = {
      'none': 1.0,
      'low': 0.85,
      'medium': 0.65,
      'high': 0.4
}


def validate_inputs(age, localisation, wound_type, comorbidities):
    """Create a list of errors and raise an exception if there are any."""

    errors = []

    # verify the age is well beetwin 0 and 120
    if not (0 <= age <= 120):
        errors.append(f"Age '{age}' is not beetwin 0:120")

    # verify the location is known
    loc_key = localisation.lower().strip()
    if loc_key not in LOCALISATION_COEFFICIENTS:
        errors.append(f"Unknown localisation: '{localisation}'")

    # verify the wound type is known
    wound_key = wound_type.lower().strip()
    if wound_key not in HEALING_SPEED_TABLE:
        errors.append(f"Unknown wound type: '{wound_type}'")

    # verify all of the comorbidities are known
    if isinstance(comorbidities, str):
        comorbidity_list = [c.strip().lower() for c in comorbidities.split(',')]
    else:
        comorbidity_list = []

    for condition in comorbidity_list:
        if condition not in known_comorbidities:
            errors.append(f"Unknown comorbidity: '{condition}'")

    # raise error if any are found and say them
    if errors:
        raise ValueError("Input validation errors:\n" + "\n".join(errors))


## C - Create maks and calculate the area


In [11]:
def create_masks(results, H, W, margin=300):
    """Create masks for wound (class 1), the surrounding of the wound, and reference (class 0)."""

    mask_wound = np.zeros((H, W), dtype=np.uint8)
    margin_mask_wound = np.zeros((H, W), dtype=np.uint8)
    mask_reference = np.zeros((H, W), dtype=np.uint8)
    mask_wound_with_margin = np.zeros((H, W), dtype=np.uint8)
    wound_box = None

    for result in results:
        if result.masks is None:
            print("no mask found.")
            continue

        masks = result.masks.data  # shape: (N, H, W)
        boxes = result.boxes.data.cpu().numpy()  # shape: (N, 6)
        clss = boxes[:, 5].astype(int)

        print("class found:", clss)

        # find wound and reference
        indices_wound = np.where(clss == 1)[0]
        indices_reference = np.where(clss == 0)[0]

        # get the wound size
        if indices_wound.size > 0:
            masks_wound = masks[indices_wound]
            mask_wound = torch.any(masks_wound, dim=0).int().cpu().numpy() * 255
            wound_box = boxes[indices_wound[0]][:4].astype(int)

            mask_binary = (mask_wound > 0).astype(np.uint8)

            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (margin,margin))
            dilated = cv2.dilate(mask_binary, kernel, iterations=1)

            margin_only = cv2.subtract(dilated, mask_binary)
            margin_mask_wound = margin_only * 255

            #print("mask_wound shape:", mask_wound.shape, "dtype:", mask_wound.dtype)
            #print("margin_mask_wound shape:", margin_mask_wound.shape, "dtype:", margin_mask_wound.dtype)
            #print("mask_wound unique values:", np.unique(mask_wound))
            #print("margin_mask_wound unique values:", np.unique(margin_mask_wound))


            if mask_wound.shape != margin_mask_wound.shape:
                margin_mask_wound = cv2.resize(margin_mask_wound, (mask_wound.shape[1], mask_wound.shape[0]), interpolation=cv2.INTER_NEAREST)

            if mask_wound.dtype != np.uint8:
                mask_wound = mask_wound.astype(np.uint8)
            if margin_mask_wound.dtype != np.uint8:
                margin_mask_wound = margin_mask_wound.astype(np.uint8)

            mask_wound = ((mask_wound > 0) * 255).astype(np.uint8)
            margin_mask_wound = ((margin_mask_wound > 0) * 255).astype(np.uint8)

            #print("After processing:")
            #print("mask_wound shape:", mask_wound.shape, "dtype:", mask_wound.dtype)
            #print("margin_mask_wound shape:", margin_mask_wound.shape, "dtype:", margin_mask_wound.dtype)
            #print("mask_wound unique values:", np.unique(mask_wound))
            #print("margin_mask_wound unique values:", np.unique(margin_mask_wound))

            mask_wound_with_margin = cv2.bitwise_or(mask_wound, margin_mask_wound)

        # get the reference size
        if indices_reference.size > 0:
            masks_reference = masks[indices_reference]
            mask_reference = torch.any(masks_reference, dim=0).int().cpu().numpy() * 255

    # Resize reference mask if needed
    if mask_reference.shape != mask_wound.shape:
        print(f"Resizing mask_reference from {mask_reference.shape} to {mask_wound.shape}")
        mask_reference = cv2.resize(mask_reference, (mask_wound.shape[1], mask_wound.shape[0]), interpolation=cv2.INTER_NEAREST)

    valid_masks = np.logical_and(mask_wound > 0, mask_reference > 0)

    return mask_wound, mask_reference, wound_box, valid_masks, margin_mask_wound, mask_wound_with_margin


def calculate_area(mask_wound, mask_reference, area_reference):
    """Calculate the actual wound area using the reference area."""

    pixel_area_wound = np.sum(mask_wound) / 255
    pixel_area_reference = np.sum(mask_reference) / 255

    pixel_area_wound = float(pixel_area_wound)
    pixel_area_reference = float(pixel_area_reference)

    # if the refence is found calculate the size from the reference
    if pixel_area_reference > 0:
        converted_area = pixel_area_wound * area_reference / pixel_area_reference
        return round(converted_area, 3)

    # if no reference found say it
    else:
        raise ValueError("No reference detected!")

## D - Find infection and necrosis traces

In [12]:
def find_infection_and_necrosis(wound_box, image_bgr, mask_wound_with_margin, mask_wound):
    """Détecte des traces d'infection (rouge, jaune) dans la région de la plaie et la zone proche"""

    x, y, w, h = wound_box


    # combined zone (wound + marging)
    combined_mask = np.logical_or(mask_wound, mask_wound_with_margin).astype(np.uint8)
    region_mask = combined_mask[y:y+h, x:x+w] > 0  # booléen

    # conversion of the image in HSV
    image_hsv = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HSV)

    # extract the wound area
    roi = image_hsv[y:y+h, x:x+w]

    # detection of red and make a mask for it

    "excluding bright red (fresh blood) and dark red (dry blood) to keep only 'medium' red"
    lower_red1 = np.array([0, 40, 120])
    upper_red1 = np.array([10, 100, 180])

    lower_red2 = np.array([170, 40, 120])
    upper_red2 = np.array([180, 100, 180])


    mask_red1 = cv2.inRange(roi, lower_red1, upper_red1)
    mask_red2 = cv2.inRange(roi, lower_red2, upper_red2)
    mask_red = cv2.bitwise_or(mask_red1, mask_red2)

    # detection of yellow and make a mask for it

    "excluding beige and yellow green and keeping yellow"
    lower_yellow = np.array([22, 80, 100])
    upper_yellow = np.array([35, 255, 255])
    mask_yellow = cv2.inRange(roi, lower_yellow, upper_yellow)

    # detection of dark / brown and make a mask for it

    "excluding light brown and dark red and keeding black and dark brown"
    lower_necrosis_brown = np.array([10, 30, 0])
    upper_necrosis_brown = np.array([20, 100, 40])

    lower_necrosis_black = np.array([0, 0, 0])
    upper_necrosis_black = np.array([180, 80, 20])

    mask_necrosis1 = cv2.inRange(roi, lower_necrosis_brown, upper_necrosis_brown)
    mask_necrosis2 = cv2.inRange(roi, lower_necrosis_black, upper_necrosis_black)
    mask_necrosis = cv2.bitwise_or(mask_necrosis1, mask_necrosis2)


    # Convert masks to bool for indexing
    mask_red = mask_red.astype(bool)
    mask_yellow = mask_yellow.astype(bool)
    mask_necrosis = mask_necrosis.astype(bool)

    # Resize region_mask if needed
    if region_mask.shape != mask_red.shape:
        print("Resizing region_mask to match mask_red")
        region_mask_resized = cv2.resize(region_mask.astype(np.uint8),(mask_red.shape[1], mask_red.shape[0]), interpolation=cv2.INTER_NEAREST).astype(bool)
    else:
        region_mask_resized = region_mask

    region_pixels = np.count_nonzero(region_mask_resized)

    if region_pixels == 0:
        return 0.0, 0.0, 0.0, 0.0

    # Convert boolean masks to uint8
    mask_red = (mask_red > 0).astype(np.uint8) * 255
    mask_yellow = (mask_yellow > 0).astype(np.uint8) * 255
    mask_necrosis = (mask_necrosis > 0).astype(np.uint8) * 255
    region_mask_resized = (region_mask_resized > 0).astype(np.uint8) * 255

    # Apply region mask to limit analysis
    mask_red = cv2.bitwise_and(mask_red, region_mask_resized)
    mask_yellow = cv2.bitwise_and(mask_yellow, region_mask_resized)
    mask_necrosis = cv2.bitwise_and(mask_necrosis, region_mask_resized)

    # create a mask with infection and necrosis to see where they are
    mask_infection = cv2.bitwise_or(mask_red, mask_yellow)
    mask_total = cv2.bitwise_or(mask_infection, mask_necrosis)

    # Calculate the are of infection and necrosis
    red_pct = (cv2.countNonZero(mask_red.astype(np.uint8)) / region_pixels) * 100
    yellow_pct = (cv2.countNonZero(mask_yellow.astype(np.uint8)) / region_pixels) * 100
    necrosis_pct = (cv2.countNonZero(mask_necrosis.astype(np.uint8)) / region_pixels) * 100

    infection_score = np.clip((red_pct * 0.5 + yellow_pct * 0.5) / 100, 0, 1.0)

    # Round final results
    return round(red_pct, 2), round(yellow_pct, 2), round(infection_score, 2), round(necrosis_pct, 2), region_mask_resized

## E - Estimate the healing time


In [13]:
def estimate_healing(surface_cm2, age, necrosis_score, wound_type, localisation, comorbidities, infection_score, HEALING_SPEED_TABLE, LOCALISATION_COEFFICIENTS,age_coef_min, age_coef_max, known_comorbidities, INFECTION_LEVELS, NECROSIS_LEVELS) :
    """Estimate healing time based on the wound parameter."""

    infection_coef = 1.0
    necrosis_coef = 1.0
    comorbidities_coef = 1.0
    loc_coef = 1.0

    # age coefficients
    def get_age_coef(age, table):
        for (low, high), coef in table.items():
            if low <= age <= high:
                return coef

    age_coef_min = get_age_coef(age, age_coef_min)
    age_coef_max = get_age_coef(age, age_coef_max)

    # Wound type coefficients
    wound_type_key = wound_type.lower().strip()
    ws_base, sd_base = HEALING_SPEED_TABLE[wound_type_key]
    ws_base = max(ws_base, 0.01) # ws_base represents the base healing speed
    sd_base = max(sd_base, 0.0) # sd_base represents the base standard deviation of healing speed

    # necrosis coefficients
    def get_necrosis_level(necrosis_score):
      if necrosis_score < 0.2:
        return 'none'
      elif necrosis_score < 0.4:
        return 'low'
      elif necrosis_score < 0.6:
        return 'medium'
      else:
        return 'high'

    necrosis_level = get_necrosis_level(necrosis_score)
    necrosis_coef = NECROSIS_LEVELS[necrosis_level]

    # Infection coefficients
    def get_infection_level(infection_score):
      if infection_score < 0.2:
        return 'none'
      elif infection_score < 0.4:
        return 'low'
      elif infection_score < 0.6:
        return 'medium'
      else:
        return 'high'

    infection_level = get_infection_level(infection_score)
    infection_coef = INFECTION_LEVELS[infection_level]

    # comorbidities coefficients
    if isinstance(comorbidities, str):
        comorbidity_list = [c.strip().lower() for c in comorbidities.split(',')]
    elif isinstance(comorbidities, list):
        comorbidity_list = [str(c).strip().lower() for c in comorbidities]
    else:
        comorbidity_list = []

    for condition in comorbidity_list:
      coef = known_comorbidities.get(condition, None)
      if coef is not None:
        comorbidities_coef *= coef

    # localisation coefficients
    loc_key = localisation.lower().strip()
    loc_coef = LOCALISATION_COEFFICIENTS[loc_key]

    # Total coefficients
    total_coef_min = infection_coef * necrosis_coef * loc_coef * age_coef_min * comorbidities_coef
    total_coef_max = infection_coef * necrosis_coef * loc_coef * age_coef_max * comorbidities_coef

    # Final adjusted wound speeds
    ws_min = ws_base * total_coef_min
    ws_max = ws_base * total_coef_max

    # Estimate healing time in days without standard deviation for now
    time_min = math.ceil(surface_cm2 / ws_max )
    time_max = math.ceil(surface_cm2 / ws_min )

    return time_max, time_min

## F - Annotate the image

In [14]:
def annotate_image(img, mask_wound, wound_box, actual_wound_area, age, necrosis_score, wound_type, localisation, comorbidities, red_pct, yellow_pct, infection_score, region_mask_resized, margin_wound_mask):
    """Annotate the image with the bounding box, wound area, and estimated healing time."""

    # Reshape the image if needed
    if mask_wound.shape != img.shape[:2]:
        print(f"reshaping of mask_wound {mask_wound.shape} to {img.shape[:2]}")
        mask_wound = cv2.resize(mask_wound, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)

    if region_mask_resized.shape != img.shape[:2]:
        print(f"reshaping of margin_mask_wound {region_mask_resized.shape} to {img.shape[:2]}")
        region_mask_resized = cv2.resize(region_mask_resized, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)

    if margin_wound_mask.shape != img.shape[:2]:
        print(f"reshaping of margin_mask_wound {margin_wound_mask.shape} to {img.shape[:2]}")
        margin_wound_mask = cv2.resize(margin_wound_mask, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Call estimate healing to have all the parameter
    time_max, time_min = estimate_healing(actual_wound_area, age, necrosis_score, wound_type, localisation, comorbidities, infection_score, HEALING_SPEED_TABLE, LOCALISATION_COEFFICIENTS,age_coef_min, age_coef_max, known_comorbidities, INFECTION_LEVELS, NECROSIS_LEVELS)

    # Create a colored mask for the wound
    colored_mask = np.zeros_like(img)
    colored_mask[:, :, 1] = mask_wound  # Use the green channel for the mask

    # Blend the original image with the colored mask
    alpha = 0.5
    img = cv2.addWeighted(img, 1, colored_mask, alpha, 0)

    if wound_box is not None:
        x1, y1, x2, y2 = wound_box

        # text
        lines = [
            f"Wound: {actual_wound_area:.1f} cm^2",
            f"Healing Time: {time_min} to {time_max} days",
            f"Age: {age}",
            f"Infection: {infection_score:.2f}",
            f"Red (inflammation): {red_pct:.1f}%, Yellow (exudate): {yellow_pct:.1f}%",
            f"Necrosis: {necrosis_score:.2f}",
            f"Wound type: {wound_type}",
            f"Location: {localisation}",
            f"Comorbidities: {comorbidities}"
        ]

        # style of the text
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        color = (255, 0, 255)
        thickness = 1
        line_height = 15
        img_h, img_w = img.shape[:2]
        text_y = img_h - (len(lines) * line_height + 10)

        # write the text into the images
        for i, line in enumerate(lines):
            cv2.putText(img, line, (100, text_y + i * line_height), font, font_scale, color, thickness)

    if region_mask_resized is not None:
        x1, y1, x2, y2 = wound_box

        mask = np.zeros_like(img)
        mask[:, :, 2] = region_mask_resized
        img = cv2.addWeighted(img, 1, mask, alpha, 0)

    if margin_wound_mask is not None:
        x1, y1, x2, y2 = wound_box

        mask = np.zeros_like(img)
        mask[:, :, 0] = margin_wound_mask
        img = cv2.addWeighted(img, 1, mask, alpha, 0)

    return img


## G - Save the image and main (choose beetwin yolo or resnet)



In [16]:
def save_image(img, output_path):
    """Save the annotated image to the specified path."""

    success = cv2.imwrite(output_path, img)
    if success:
        print(f"Image saved successfully to {output_path}")
    else:
        print(f"Failed to save image to {output_path}")

def main(input_folder, path_model, area_reference, output_folder):

    # first load the model and the files
    # model = load_model_resnet(path_model)
    model = load_model_yolo(path_model)
    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for image_file in image_files:
        path_img = os.path.join(input_folder, image_file)

        img, H, W = load_image(path_img)

        # segment the image
        masks = segment_image_yolo(model, img)
        # masks = segment_image_resnet(model, img)

        #create the mask
        mask_wound, mask_reference, wound_box, valid_masks, margin_mask_wound, mask_wound_with_margin = create_masks(masks, H, W, margin=50)

        if not np.any(mask_wound):
            print(f"{image_file}: no wound — skipping.")
            continue
        if not np.any(mask_reference):
            print(f"{image_file}: no reference — skipping.")
            continue

        # ask the parameter
        age, wound_type, localisation, comorbidities = imput_parameter(image_file)

        # verify it
        validate_inputs(age, localisation, wound_type, comorbidities)

        try:
            # calculate the area
            actual_wound_area = calculate_area(mask_wound, mask_reference, area_reference)

            # find infection and necrosis
            red_pct, yellow_pct, infection_score, necrosis_score, region_mask_resized = find_infection_and_necrosis(wound_box, img, mask_wound_with_margin, mask_wound)

            # annotate the image
            annotated_img = annotate_image(img, mask_wound, wound_box, actual_wound_area, age, necrosis_score, wound_type, localisation, comorbidities, red_pct, yellow_pct, infection_score, region_mask_resized, margin_mask_wound)

            # save the image
            output_path = os.path.join(output_folder, f"{os.path.splitext(image_file)[0]}_annotated.jpg")
            save_image(annotated_img, output_path)

        except ValueError as e:
            print(f"{image_file}: {e}")

## H - Make the estimation


In [18]:
input_folder = "/content/gdrive/MyDrive/wound_management/data_with_ref/test"
path_model = "/content/gdrive/MyDrive/wound_management/basic_wound_management_yolov11n-seg.pt"
area_reference = 9.9225
output_folder = "/content/gdrive/MyDrive/wound_management/healing/healing_n/"

# make sure the output file exist and cerate it if necessary
if not os.path.exists(output_folder):
    print("output file created")
    os.makedirs(output_folder)
else:
    print("output file already here")

# run the image processing
main(input_folder, path_model, area_reference, output_folder)

output file already here

0: 640x640 1 reference, 1 wound, 702.5ms
Speed: 10.6ms preprocess, 702.5ms inference, 42.0ms postprocess per image at shape (1, 3, 640, 640)
class found: [0 1]
How old is the patient (CARLOS ARTUROIMG576.jpg)? 81
What is the wound type (CARLOS ARTUROIMG576.jpg)? (e.g., skintear, burn, diabetic foot): burn
Where is the wound (CARLOS ARTUROIMG576.jpg)? (e.g., feet, heels, legs): feet
Any other conditions (CARLOS ARTUROIMG576.jpg)? (e.g., diabete, smoking, hiv, can be more than one): hiv
reshaping of margin_mask_wound (374, 347) to (640, 640)
Image saved successfully to /content/gdrive/MyDrive/wound_management/healing/healing_n/CARLOS ARTUROIMG576_annotated.jpg

0: 640x640 1 reference, 1070.1ms
Speed: 7.6ms preprocess, 1070.1ms inference, 12.2ms postprocess per image at shape (1, 3, 640, 640)
class found: [0]
FABIOIMG1406.jpg: no wound — skipping.

0: 640x640 2 references, 247.3ms
Speed: 4.1ms preprocess, 247.3ms inference, 8.4ms postprocess per image at shape (1

ValueError: invalid literal for int() with base 10: 'a'

In [ ]:
input_folder2 = "/content/gdrive/MyDrive/wound_management/data_with_ref/test"
path_model2 = "/content/gdrive/MyDrive/wound_management/basic_wound_management_yolov11x-seg.pt"
area_reference = 9.9225
output_folder2 = "/content/gdrive/MyDrive/wound_management/healing/healing_x/"

# make sure the output file exist and cerate it if necessary
if not os.path.exists(output_folder2):
    print("output file created")
    os.makedirs(output_folder2)
else:
    print("output file already here")

# run the image processing
main(input_folder2, path_model2, area_reference, output_folder2)

output file already here

0: 640x640 1 reference, 3 wounds, 9631.4ms
Speed: 6.0ms preprocess, 9631.4ms inference, 20.7ms postprocess per image at shape (1, 3, 640, 640)
class found: [0 1 1 1]
How old is the patient SIMATECIMAG61.jpg? 47
SIMATECIMAG61.jpg: Actual wound area: 20.951 cm²
Image saved successfully to /content/gdrive/MyDrive/wound_management/healing_x/SIMATECIMAG61_annotated.jpg

0: 640x640 1 reference, 4 wounds, 7804.6ms
Speed: 5.3ms preprocess, 7804.6ms inference, 36.3ms postprocess per image at shape (1, 3, 640, 640)
class found: [0 1 1 1 1]
How old is the patient SIMATECIMAG02.jpg? 82
SIMATECIMAG02.jpg: Actual wound area: 5.23 cm²
Image saved successfully to /content/gdrive/MyDrive/wound_management/healing_x/SIMATECIMAG02_annotated.jpg

0: 640x640 1 reference, 2 wounds, 5909.0ms
Speed: 4.2ms preprocess, 5909.0ms inference, 23.1ms postprocess per image at shape (1, 3, 640, 640)
class found: [0 1 1]
How old is the patient CARLOS ARTUROIMG561.jpg? 65
CARLOS ARTUROIMG561.jpg

In [ ]:
input_folder = "/content/gdrive/MyDrive/wound_management/data_with_ref/test"
path_model = "/content/gdrive/MyDrive/wound_management/wound_management_deeplabv3.pth"
area_reference = 9.9225
output_folder = "/content/gdrive/MyDrive/wound_management/healing/healing_50/"

# make sure the output file exist and cerate it if necessary
if not os.path.exists(output_folder):
    print("output file created")
    os.makedirs(output_folder)
else:
    print("output file already here")

# run the image processing
main(input_folder, path_model, area_reference, output_folder)

output file already here


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 157MB/s]


RuntimeError: Error(s) in loading state_dict for DeepLabV3:
	size mismatch for classifier.4.weight: copying a param with shape torch.Size([1, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([2, 256, 1, 1]).
	size mismatch for classifier.4.bias: copying a param with shape torch.Size([1]) from checkpoint, the shape in current model is torch.Size([2]).
	size mismatch for aux_classifier.4.weight: copying a param with shape torch.Size([1, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([2, 256, 1, 1]).
	size mismatch for aux_classifier.4.bias: copying a param with shape torch.Size([1]) from checkpoint, the shape in current model is torch.Size([2]).

# VI - Temperature




## A - adding data and annotate it

In [ ]:
# annotate the dataset

# create data3.yaml

# add a file data_thermal

## B - yolov11n-seg for thermal imaging

In [ ]:
"""Load the model"""

model3 = YOLO("/content/gdrive/MyDrive/wound_management/wound_model_yolov11x-seg.pt")

In [ ]:
"""Train the model"""

model3.train( data="/content/gdrive/MyDrive/wound_management/data3.yaml", epochs=250, batch=-1, patience=10)

In [ ]:
"""Record the model"""

detect_path = Path("runs/segment")

# Find the actual train version
train_dirs = sorted(detect_path.glob("train*"), key=os.path.getmtime)

if not train_dirs:
    print(" Aucun dossier 'train*' trouvé dans runs/segment.")
else:
    latest_train_dir = train_dirs[-1]
    best_pt_path = latest_train_dir / "weights" / "best.pt"

    if best_pt_path.exists():
        # Copy it in google drive
        dest_path = "/content/gdrive/MyDrive/wound_management/thermal_wound_management_yolov11x-seg.pt"
        shutil.copy(best_pt_path, dest_path)
        print(f"Model copy from : {best_pt_path} in : {dest_path}")
    else:
        print(f"file not found {latest_train_dir}")

In [ ]:
"""Validate the model"""

wound_thermal = YOLO("/content/gdrive/MyDrive/wound_management/thermal_wound_management_yolov11x-seg.pt")
wound_thermal = wound_thermal.val(data="/content/gdrive/MyDrive/wound_management/data3.yaml")

print("mAP50:", wound_thermal.seg.map50)
print("mAP50-95:", wound_thermal.seg.map)
print("Precision:", wound_thermal.seg.p)
print("Recall:", wound_thermal.seg.r)

In [ ]:
"""Made a prediction with the model"""

wound_thermal = YOLO("/content/gdrive/MyDrive/wound_management/thermal_wound_management_yolov11x-seg.pt")

results = wound_thermal.predict(source="/content/gdrive/MyDrive/wound_management/data_thermal/test/", conf=0.75)
total_images = len(results)
missing_class_0 = 0
missing_class_1 = 0
missing_both = 0

for r in results:
    if not r.boxes or len(r.boxes.cls) == 0:
        missing_both += 1
        missing_class_0 += 1
        missing_class_1 += 1
        continue

    detected_classes = [int(cls_id) for cls_id in r.boxes.cls]

    has_class_0 = 0 in detected_classes
    has_class_1 = 1 in detected_classes

    if not has_class_0:
        missing_class_0 += 1
    if not has_class_1:
        missing_class_1 += 1
    if not has_class_0 and not has_class_1:
        missing_both += 1

print(f"Images without reference : {missing_class_0} ({missing_class_0 / total_images:.2%})")
print(f"Images without wound : {missing_class_1} ({missing_class_1 / total_images:.2%})")
print(f"Nothing found : {missing_both} ({missing_both / total_images:.2%})")

In [ ]:
"""Make a prediction and save the results"""

results = wound_management.predict(source="/content/gdrive/MyDrive/wound_management/data_thermal/test/", conf=0.25, save=True,
                        project="/content/gdrive/MyDrive/wound_management/output_x-seg/",
                        name="predict_thermal" )

## C - Estimate the healing time

In [ ]:
input_folder = "/content/gdrive/MyDrive/wound_management/data_thermal/test"
path_model = "/content/gdrive/MyDrive/wound_management/thermal_wound_management_yolov11x-seg.pt"
area_reference = 9.9225
output_folder = "/content/gdrive/MyDrive/wound_management/healing_x/"

# make sure the output file exist and cerate it if necessary
if not os.path.exists(output_folder):
    print("output file created")
    os.makedirs(output_folder)
else:
    print("output file already here")

# run the image processing
main(input_folder, path_model, area_reference, output_folder)